# Set up Environment

## Imports

In [ ]:
import os
os.environ["SCIPY_ARRAY_API"] = "1"

In [ ]:
## OS and Operations
import os
import re
import ast
import json
import time
import base64
import string
import unicodedata

# Data
import numpy as np
import pandas as pd
from collections import defaultdict, Counter

# Vizualisation
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.graphics.mosaicplot import mosaic

# ML
import torch
import tensorflow as tf
from transformers import CamembertTokenizer, CamembertForTokenClassification, pipeline

# Statistic analysis
from scipy import stats
from scipy.stats import (
    spearmanr,
    chi2_contingency,
    pearsonr,
    kendalltau,
    mannwhitneyu,
    entropy
)
import statsmodels.api as sm
from patsy import dmatrix
from sklearn.utils import resample
from sklearn.metrics import (
    mutual_info_score,
    normalized_mutual_info_score,
    adjusted_mutual_info_score,
)

# NLP
from unidecode import unidecode
from rapidfuzz import fuzz, process
import stanza
import fitz
    
# LLMs
import openai
import anthropic
from anthropic.types.messages.batch_create_params import Request
from anthropic.types.message_create_params import MessageCreateParamsNonStreaming

# Display
from IPython.display import display, clear_output
from moviepy import ImageSequenceClip
import warnings

from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss, roc_auc_score

import xgboost as xgb

from sklearn.metrics import (
    accuracy_score, recall_score, f1_score,
    confusion_matrix, ConfusionMatrixDisplay, classification_report
)

from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import make_scorer
from sklearn.linear_model import LogisticRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
import shap
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)

## Paths, Data, Global Functions

In [ ]:
OUT_PATH="/Users/antonioslagarias/Documents/OFF/Exports/adho"

In [ ]:
FIG_PATH="/Users/antonioslagarias/Documents/OFF/Exports/adho/figures"

In [ ]:
def clean_json_string(gpt_output):
    if not isinstance(gpt_output, str):
        return None

    # Extract content between first '{' and last '}'
    match = re.search(r"\{.*\}", gpt_output, re.DOTALL)
    if match:
        return match.group(0).strip()
    return None  # Return None if no valid JSON found

In [ ]:
def normalize_text(text):
    if not isinstance(text, str):
        return ''
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ascii', errors='ignore').decode('utf-8')
    return text.strip()

In [ ]:
file="/Users/antonioslagarias/Documents/OFF/Exports/data_2013/festival_data_draft_2013.xlsx"  
data_draft=pd.read_excel(file)

data_draft.columns

# Import Data after Scrapping

See separate notebook for the code used for scrapping from Officiel des Spectacles

In [ ]:
file="/Users/antonioslagarias/Documents/OFF/Exports/adho/offi_enriched_2013_nona.csv"  
data_offi=pd.read_csv(file)

In [ ]:
data_offi.head(5)

In [ ]:
file="/Users/antonioslagarias/Documents/OFF/Exports/adho/data_adho_.12072025.xlsx"  
data_draft=pd.read_excel(file)

In [ ]:
data_draft.columns

## Prepare Scrapped Data

In [ ]:
pd.reset_option("display.max_rows")

In [ ]:
print(data_offi['enriched'].apply(type).value_counts())

In [ ]:
print(f"Rows in data_draft: {len(data_draft)}")

## Get Matches with Scrapped Data

Matches are based on a fuzzy match logic. For the same creator, a much in a shows titles with a score over 55 percent is considered valid. Outliers (e.g. shows programmed 8 or more years after 2013 are mannually verified below).

In [ ]:
data_offi['enriched'] = data_offi['enriched'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [ ]:
def get_entries_with_years(enriched):
    if not isinstance(enriched, dict):
        return []
    results = []
    for year, entries in enriched.items():
        for entry in entries:
            title = entry.get('title', '').strip()
            if title:
                results.append((entry, year))  # Keep full entry
    return results

all_matches_per_row = []

for _, row in data_draft.iterrows():
    id_val = str(row['ID'])  # safe if already string
    target_title = row['title']

    enriched_row = data_offi[data_offi['ID'] == id_val]
    if enriched_row.empty:
        all_matches_per_row.append(None)
        continue

    enriched = enriched_row.iloc[0]['enriched']
    entry_year_pairs = get_entries_with_years(enriched)

    if not entry_year_pairs:
        all_matches_per_row.append(None)
        continue

    titles = [entry.get('title', '') for entry, _ in entry_year_pairs]

    matches = process.extract(target_title, titles, scorer=fuzz.token_sort_ratio, limit=None)
 
    filtered_matches = []
    for matched_title, score, idx in matches:
        if score >= 55:
            entry, year = entry_year_pairs[idx]

            # Prepare dict with all available enriched info, prefixed with 'offi_'
            match_dict = {
                'match_score': score,
                'matched_title': matched_title,
                'offi_year': year
            }

            for k, v in entry.items():
                match_dict[f'offi_{k}'] = v

            filtered_matches.append(match_dict)

    all_matches_per_row.append(filtered_matches if filtered_matches else None)

data_draft['all_matches'] = all_matches_per_row

In [ ]:
def normalize_title(text):
    """Lowercase, remove accents, punctuation, and collapse whitespace."""
    if not isinstance(text, str):
        return ''
    text = unidecode(text.lower())
    text = re.sub(r'[^\w\s]', '', text)   # Remove punctuation
    text = re.sub(r'\s+', ' ', text)      # Normalize spaces
    return text.strip()

def get_entries_with_years(enriched):
    if not isinstance(enriched, dict):
        return []
    results = []
    for year, entries in enriched.items():
        for entry in entries:
            title = entry.get('title', '').strip()
            if title:
                results.append((entry, year))
    return results

all_matches_per_row = []

for _, row in data_draft.iterrows():
    id_val = str(row['ID'])
    target_title = row['title']
    target_norm = normalize_title(target_title)

    enriched_row = data_offi[data_offi['ID'] == id_val]
    if enriched_row.empty:
        all_matches_per_row.append(None)
        continue

    enriched = enriched_row.iloc[0]['enriched']
    entry_year_pairs = get_entries_with_years(enriched)

    if not entry_year_pairs:
        all_matches_per_row.append(None)
        continue

    raw_titles = [entry.get('title', '') for entry, _ in entry_year_pairs]
    norm_titles = [normalize_title(t) for t in raw_titles]

    matches = process.extract(target_norm, norm_titles, scorer=fuzz.token_sort_ratio, limit=None)

    filtered_matches = []
    for matched_norm, score, idx in matches:
        if score >= 55:
            entry, year = entry_year_pairs[idx]
            match_dict = {
                'match_score': score,
                'matched_title': raw_titles[idx],
                'offi_year': year
            }
            for k, v in entry.items():
                match_dict[f'offi_{k}'] = v
            filtered_matches.append(match_dict)

    all_matches_per_row.append(filtered_matches if filtered_matches else None)

data_draft['all_matches'] = all_matches_per_row

In [ ]:
data_draft['all_matches'].isna().sum()

Create a new dataframe to examine shows reprogrammed in multiple years.

In [ ]:
# Step 1: Explode the all_matches column
df_exploded = data_draft.explode('all_matches', ignore_index=True)

# Step 2: Drop rows with no match
df_exploded = df_exploded[df_exploded['all_matches'].notna()].copy()

# Step 3: Normalize the dictionary in each match into columns
match_details = pd.json_normalize(df_exploded['all_matches'])

# Step 4: Combine into full exploded DataFrame
df_exploded = df_exploded.drop(columns=['all_matches']).reset_index(drop=True)
df_exploded = pd.concat([df_exploded, match_details], axis=1)

In [ ]:
df_exploded['API_names_matched'] = np.nan
df_exploded['API_names_found'] = np.nan

Further filtering the matches. Now matches are accepted only when at least one name of a collaborators are identified within the already matched shows based on the creator and the title

In [ ]:
def extract_names_from_api_roles(api_roles):
    if pd.isna(api_roles):
        return []

    try:
        if isinstance(api_roles, str):
            roles_dict = json.loads(api_roles)
        elif isinstance(api_roles, dict):
            roles_dict = api_roles
        else:
            return []
    except Exception:
        return []

    names = []
    for role_list in roles_dict.values():
        if isinstance(role_list, list):
            names.extend(role_list)
    return [normalize_text(name) for name in names if isinstance(name, str)]

# Apply matching
matched_names = []
has_match_flags = []

for _, row in df_exploded.iterrows():
    offi_roles = normalize_text(row.get('offi_roles', ''))
    api_names = extract_names_from_api_roles(row.get('API_roles', {}))

    if offi_roles:
        matched = [name for name in api_names if name in offi_roles]
    else:
        matched = []  # ❗ ensure matched is always defined

    matched_names.append(matched if matched else None)
    has_match_flags.append(bool(matched))

# Add results to DataFrame
df_exploded['API_names_matched'] = matched_names
df_exploded['API_names_found'] = has_match_flags

In [ ]:
true_count = df_exploded['API_names_found'].sum()
print(f"Number of rows with at least one matched API name: {true_count}")

In [ ]:
id_with_match_count = df_exploded[df_exploded['API_names_found']].ID.nunique()
print(f"Number of unique IDs with at least one API name match: {id_with_match_count}")

Manual verification for shows that have no casting information or appear as outliers

In [ ]:
df_exploded_filtered = df_exploded[
    df_exploded['match_score'].between(55, 80, inclusive='left') &
    (df_exploded['API_names_found'] == True)
].copy()
df_exploded_filtered.to_excel("shows_to_verify.xlsx", index=False)

Set further limits after manual verification

In [ ]:
# Ensure year is integer
df_exploded['offi_year'] = df_exploded['offi_year'].astype(int)

# Apply filtering rules
df_exploded = df_exploded[
    (
        ((df_exploded['offi_year'].between(2010, 2017)) & (df_exploded['match_score'] > 57))
    )
    |
    ((df_exploded['offi_year'] > 2017) & (df_exploded['match_score'] > 75))
].copy()

In [ ]:
pd.set_option("display.max_rows", None)

In [ ]:
# Normalize offi_roles to detect empty entries
df_exploded['offi_roles_normalized'] = df_exploded['offi_roles'].apply(normalize_text)
df_exploded['offi_title_normalized'] = df_exploded['offi_title'].apply(normalize_text)

# Determine where roles is empty but name matched in title
step4_matches = []

for _, row in df_exploded.iterrows():
    if row['offi_roles_normalized']:  # Skip if roles is non-empty
        step4_matches.append(False)
        continue

    api_names = extract_names_from_api_roles(row.get('API_roles', {}))
    title_text = row['offi_title_normalized']

    matched = any(name in title_text for name in api_names)
    step4_matches.append(matched)

# Add a column flag to identify Step 4 logic
df_exploded['matched_from_title'] = step4_matches

# Filter only those that matched via Step 4
df_step4_only = df_exploded[df_exploded['matched_from_title']]

# Show selected columns for inspection
df_step4_only[['ID', 'title', 'offi_title', 'API_roles', 'API_names_matched', 'offi_roles','API_names_found', "offi_year"]]

In [ ]:
df_step4_only = df_exploded[df_exploded['matched_from_title']].copy()
df_step4_only['step4_keep'] = False  # default, to be manually changed
indexes_to_keep = [59, 313, 314, 315, 332, 348, 349, 350, 364, 461, 473, 595, 664, 665, 738, 769, 888, 889, 892, 930]
df_step4_only.loc[indexes_to_keep, 'step4_keep'] = True

In [ ]:
df_exploded = df_exploded.merge(
    df_step4_only[['step4_keep']],
    left_index=True,
    right_index=True,
    how='left'
)

# Fill missing values with False
df_exploded['step4_keep'] = df_exploded['step4_keep'].fillna(False)

In [ ]:
df_exploded.columns

In [ ]:
df_exploded=df_exploded.drop(columns=['offi_roles_normalized',
       'offi_title_normalized', 'matched_from_title'])

In [ ]:
df_exploded['reprog'] = df_exploded['API_names_found'] | df_exploded['step4_keep']

In [ ]:
true_count = df_exploded['reprog'].sum()
print(f"Number of rows with at least one matched API name: {true_count}")

In [ ]:
id_with_match_count = df_exploded[df_exploded['reprog']].ID.nunique()
print(f"Number of unique IDs with at least one reprog match: {id_with_match_count}")

In [ ]:
ids_with_reprog = df_exploded.loc[df_exploded['reprog'], 'ID'].unique()
data_draft['reprog'] = data_draft['ID'].isin(ids_with_reprog)

In [ ]:
id_with_match_count = data_draft[data_draft['reprog']].ID.nunique()
print(f"Number of unique IDs with at least one reprog match: {id_with_match_count}")

In [ ]:
id_unique = data_draft['ID'].nunique()
print(f"Number of unique IDs: {id_unique}")

In [ ]:
per=id_with_match_count/id_unique*100
print(f"{round(per, 2)}%")

# Descriptives and Correlation

## Global Descriptives on Repogramming Rate

In [ ]:
df_reprog = df_exploded[df_exploded['reprog']].copy()

# Normalize year to int
df_reprog['offi_year'] = df_reprog['offi_year'].astype(int)

# Aggregate per ID
reprog_summary = (
    df_reprog
    .groupby('ID')
    .agg(
        first_year=('offi_year', 'min'),
        all_years=('offi_year', lambda x: sorted(set(x))),
        count_years=('offi_year', 'nunique')
    )
    .reset_index()
)

# Optional: mark shows with multiple years
reprog_summary['multiple_years'] = reprog_summary['count_years'] > 1

In [ ]:
fig = px.histogram(
    reprog_summary,
    x="first_year",
    nbins=len(reprog_summary['first_year'].unique()),
    title="Number of Reprogrammed Shows by First Year",
    labels={"first_year": "First Reprog Year"},
)
fig.update_layout(xaxis=dict(tickmode='linear'))
fig.show()

In [ ]:
# For every show (ID) work out its first reprog year
first_year_per_id = (
    df_reprog.groupby('ID')['offi_year']
    .min()
    .rename('first_year')
)

df_reprog = df_reprog.merge(first_year_per_id, on='ID', how='left')

# Tag each row as “first” vs “subsequent”
df_reprog['is_first'] = df_reprog['offi_year'] == df_reprog['first_year']

# Aggregate counts by calendar year & category
year_counts = (
    df_reprog
    .groupby(['offi_year', 'is_first'])
    .size()
    .reset_index(name='n')
)

# Plot: one bar per year, coloured by first / subsequent
fig = px.bar(
    year_counts,
    x='offi_year',
    y='n',
    color='is_first',
    color_discrete_map={True: 'steelblue', False: 'orange'},
    barmode='group',
    labels=dict(offi_year='Year', n='Number of Shows', is_first='Category'),
    title='Reprogrammed Shows per Year (first vs. subsequent)'
)
fig.update_layout(xaxis=dict(tickmode='linear'))
fig.show()

In [ ]:
#Manual verification for outliers
ids_after_2018 = df_reprog.loc[df_reprog['offi_year'] > 2018, 'ID'].unique()

# Extract all rows (all years) for those shows
to_verify = df_reprog[df_reprog['ID'].isin(ids_after_2018)].copy()

# Optional: sort by ID and year for easier review
to_verify = to_verify.sort_values(['ID', 'offi_year'])

# Export to Excel
to_verify.to_excel("shows_with_year_after_2018_2.xlsx", index=False)

In [ ]:
total_ids            = data_draft['ID'].nunique()      # every show
total_ids_reprog     = df_exploded['ID'].nunique()     # shows that have ≥1 reprog

first_year_df = (
    df_reprog               # df_reprog already = df_exploded[df_exploded['reprog']]
    .groupby('ID')['offi_year']
    .min()
    .reset_index(name='first_year')
)

first_year_counts = (
    first_year_df
    .groupby('first_year')['ID']
    .nunique()
    .reset_index(name='first_year_count')
    .rename(columns={'first_year': 'year'})
)


first_year_counts['percent_all']          = (first_year_counts['first_year_count'] / total_ids        * 100).round(2)
first_year_counts['percent_reprog_only']  = (first_year_counts['first_year_count'] / total_ids_reprog * 100).round(2)

print("First-year reprogramming rates:")
print(first_year_counts[['year', 'first_year_count', 'percent_all', 'percent_reprog_only']]
      .to_string(index=False))

### Seperate pre- and post-festival shows for .

In [ ]:
# French month names (lower-case for matching)
french_months = [
    "janvier", "février", "mars", "avril", "mai", "juin",
    "juillet", "août", "septembre", "octobre", "novembre", "décembre"
]

# Build a regex that matches any of the above as whole words
month_pattern = r'\b(?:' + '|'.join(french_months) + r')\b'

# Create a new column `offi_months`
#     • For 2013 rows: list of matched months (lower-case strings)
#     • For other years: empty list
def extract_months(row):
    if row['offi_year'] != 2013 or not isinstance(row['offi_dates'], str):
        return []                       
    return re.findall(month_pattern, row['offi_dates'].lower())

df_exploded['offi_months'] = df_exploded.apply(extract_months, axis=1)

In [ ]:
# Define the order of French months
month_order = {
    "janvier": 1, "février": 2, "mars": 3, "avril": 4, "mai": 5, "juin": 6,
    "juillet": 7, "août": 8, "septembre": 9, "octobre": 10, "novembre": 11, "décembre": 12
}

# Function to determine if first month is before July
def is_pre_festival(months):
    if not months:
        return False
    first_month = months[0]
    return month_order.get(first_month, 13) < 7

# Apply to create new column
df_exploded['pro-festival'] = df_exploded['offi_months'].apply(is_pre_festival)

In [ ]:
# Filter for 2013 only
df_2013 = df_exploded[df_exploded['offi_year'] == 2013]

# Count True/False values in 'pro-festival'
festival_counts_2013 = df_2013['pro-festival'].value_counts()

# Display results
print("Pro-festival counts for 2013:")
print(festival_counts_2013)

In [ ]:
# Group by year and pro-festival flag
festival_bar_data = (
    df_exploded[df_exploded['offi_year'] == 2013]
    .groupby(['offi_year', 'pro-festival'])
    .size()
    .reset_index(name='count')
)

In [ ]:
# ── 1. data prep ────────────────────────────────────────────────────
df = df_exploded[df_exploded['reprog']].copy()

df['kind'] = np.where(
    df['offi_year'] == df.groupby('ID')['offi_year'].transform('min'),
    'First Reprog.', 'Subseq. Reprog.'
)

df['grp'] = np.where(
    df['offi_year'] == 2013,
    np.where(df['pro-festival'], 'Pro', 'NonPro'),
    'All'
)

# ── 2. aggregate counts ─────────────────────────────────────────────
agg = (df.groupby(['offi_year', 'grp', 'kind'])['ID']
         .nunique()
         .reset_index(name='n'))

# ── 3. position bars on the x-axis ──────────────────────────────────
def xpos(r):
    if r.offi_year == 2013 and r.grp != 'All':
        return r.offi_year - 0.3 if r.grp == 'Pro' else r.offi_year + 0.3
    return r.offi_year

agg['x'] = agg.apply(xpos, axis=1)

# ── 4. build the figure ─────────────────────────────────────────────
col = {'First Reprog.': 'steelblue', 'Subseq. Reprog.': 'darkorange'}
fig = go.Figure()

for kind in ['First Reprog.', 'Subseq. Reprog.']:
    d = agg[agg['kind'] == kind]
    fig.add_bar(
        x=d['x'],
        y=d['n'],
        name=kind,
        legendgroup=kind,
        offsetgroup=kind,
        marker_color=col[kind]
    )

# Vertical separator at July 2013
fig.add_shape(
    type='line',
    x0=2013, x1=2013,
    y0=0, y1=agg['n'].max() * 1.15,
    line=dict(color='black', width=3)
)

fig.update_layout(
    barmode='group',
    title='Reprogrammed Shows – split for 2013',
    xaxis=dict(
        title='Year',
        tickmode='linear',
        linecolor='black',
        mirror=False,
        showgrid=False,
        zeroline=False
    ),
    yaxis=dict(
        title='N. of Shows',
        linecolor='black',
        mirror=False,  # mirror removed here
        showgrid=False,
        zeroline=False
    ),
    plot_bgcolor='white',
    paper_bgcolor='white',
    legend=dict(
    title='',
    borderwidth=0,
    font=dict(size=10),
    x=0.84,
    y=1,
    xanchor='left',
    yanchor='top'
)
)

# Save the figure
png_path = os.path.join(FIG_PATH, "reprog_bar_split.png")
pdf_path = os.path.join(FIG_PATH, "reprog_bar_split.pdf")
fig.write_image(png_path, scale=3, width=900, height=600)
fig.write_image(pdf_path, width=900, height=600)
fig.show()

### Estimate the average life of a show 

This estimation is made based on the number fuzzy matches per show

In [ ]:
# Keep only rows with reprog == True
df_reprog = df_exploded[df_exploded['reprog']].copy()

# Compute life as number of rows per ID
life_counts = (
    df_reprog.groupby('ID')
    .size()
    .rename('life')
    .reset_index()
)

# Compute median
median_life = life_counts['life'].median()

# Plot histogram
fig_life = px.histogram(
    life_counts,
    x='life',
    histfunc='count',
    title='Distribution of Show Lifespans (Row Count)',
    labels={'life': 'Lifespan (number of rows)'},
    color_discrete_sequence=['steelblue']
)

fig_life.update_layout(
    yaxis_title='Number of Shows',
    xaxis=dict(dtick=1),
    bargap=0.1
)

# Add median line
fig_life.add_vline(
    x=median_life,
    line_dash="dash",
    line_color="crimson",
    annotation_text=f"Median: {median_life:.1f}",
    annotation_position="top right"
)

fig_life.show()

In [ ]:
 # Keep only rows with reprog == True
df_reprog = df_exploded[df_exploded['reprog']].copy()

# Compute life as number of rows per ID
life_counts = (
    df_reprog.groupby('ID')
    .size()
    .rename('life')
    .reset_index()
)

# Median lifespan
median_life = life_counts['life'].median()

# Percentage with life < 3
percent_under_3 = 100 * (life_counts['life'] < 3).sum() / len(life_counts)
percent_under_4 = 100 * (life_counts['life'] < 4).sum() / len(life_counts)

# Print results
print(f"Median lifespan: {median_life:.2f} years")
print(f"Percentage of shows with lifespan < 3 years: {percent_under_3:.2f}%")
print(f"Percentage of shows with lifespan < 4 years: {percent_under_4:.2f}%")

Export Dataframe for further analysis

In [ ]:
data_draft.head(3)

In [ ]:
date="12072025"
file = f"/Users/antonioslagarias/Documents/OFF/Exports/adho/data_adho_.{date}.xlsx"
data_draft.to_excel(file, index=False)

## Correlation

In [ ]:
file="/Users/antonioslagarias/Documents/OFF/Exports/adho/data_adho_prog.xlsx"  
data_draft=pd.read_excel(file)

In [ ]:
data_draft.columns

In [ ]:
pd.reset_option("display.max_rows")

#### Preparing the data, remove outliers

In [ ]:
# helper to count names in "Interprètes" / "Interprète" / "avec" categories
def count_interpretes(cell):
    if pd.isna(cell):
        return 0
    # try JSON → literal_eval fallback
    try:
        data = json.loads(cell)
    except Exception:
        try:
            data = ast.literal_eval(cell)
        except Exception:
            return 0
    count = 0
    for key, value in data.items():
        key_norm = key.lower()
        if any(x in key_norm for x in ("interprète", "interprete", "interprètes", "interpretes", "avec")):
            if isinstance(value, list):
                count += len(value)
            elif isinstance(value, str):
                # split simple name lists
                names = [n.strip() for n in re.split(r',|;| et ', value) if n.strip()]
                count += len(names)
    return count

# add new column with the count
data_draft["n_actors"] = data_draft["API_roles"].apply(count_interpretes)

In [ ]:
df=data_draft.copy()

In [ ]:
df["duration_minutes"] = pd.to_numeric(df["duration_minutes"], errors="coerce")
df["duration_minutes"].value_counts()

In [ ]:
df = df.drop(df[df["duration_minutes"].isin([0, 1440, 3300])].index)

In [ ]:
df["n_actors"] = pd.to_numeric(df["n_actors"], errors="coerce")

In [ ]:
df["n_actors"].value_counts()

In [ ]:
df["reprog"] = df["reprog"].astype(int)

In [ ]:
def cramers_v(table):
    """Cramér’s V."""
    chi2, _, _, _ = chi2_contingency(table)
    n = table.values.sum()
    r, k = table.shape
    return np.sqrt((chi2 / n) / (min(k - 1, r - 1)))

def cramers_v_bergsma(table):
    """Bergsma-corrected Cramér’s V"""
    chi2, _, _, _ = chi2_contingency(table)
    n = table.values.sum()
    r, k = table.shape
    phi2 = chi2 / n
    phi2_corr = max(0, phi2 - ((k - 1) * (r - 1)) / (n - 1))
    r_corr = r - ((r - 1) ** 2) / (n - 1)
    k_corr = k - ((k - 1) ** 2) / (n - 1)
    return np.sqrt(phi2_corr / min(k_corr - 1, r_corr - 1))

#### Actors

In [ ]:
# Actors
actors = (
    df.groupby("n_actors")["reprog"]
      .agg(total="count", reprog_1="sum")
      .assign(percent=lambda d: 100 * d["reprog_1"] / d["total"])
      .sort_values("percent", ascending=False)
)

actors["percent"] = actors["percent"].round(1)

pd.set_option("display.max_rows", None)  # optional: see all rows
print("🏛️ Percentage of shows programmed (reprog = 1) by number of actors:\n")
display(actors)

In [ ]:
df.loc[df["n_actors"] == 23, "title"]

In [ ]:
df_corr = df[["n_actors", "reprog"]].dropna()
df_corr = df_corr[df_corr["n_actors"] > 0]

# Pearson correlation (equivalent to point-biserial when one var is binary)
r, p = stats.pearsonr(df_corr["n_actors"], df_corr["reprog"])

# Display
print(f"Filtered rows used: {len(df_corr)}")
print(f"Point-biserial correlation r  = {r:.3f}")
print(f"p-value                        = {p:.4g}")

In [ ]:
df_corr = df[["n_actors", "reprog"]].dropna()
df_corr = df_corr[df_corr["n_actors"] > 0]

# Define feature and target
X = df_corr[["n_actors"]]  # must be 2D
y = df_corr["reprog"]

# Fit simple logistic model using only this feature
clf = LogisticRegression(solver='liblinear')
clf.fit(X, y)

# Predict probabilities
y_scores = clf.predict_proba(X)[:, 1]

# Compute AUC
auc = roc_auc_score(y, y_scores)

# Display
print(f"Filtered rows used: {len(df_corr)}")
print(f"AUC (1-feature model): {auc:.3f}")

In [ ]:
print(df["n_actors"].describe())

In [ ]:
print(df["n_actors"].value_counts().sort_index())

In [ ]:
df["n_actors"].hist(bins=20)
plt.xlabel("Number of actors")
plt.ylabel("Frequency")
plt.title("Distribution of n_actors")
plt.show()

In [ ]:
top_val = df["n_actors"].value_counts(normalize=True).iloc[0]
print(f"Top frequency proportion: {top_val:.2%}")

In [ ]:
group0 = df[df["reprog"] == 0]["n_actors"]
group1 = df[df["reprog"] == 1]["n_actors"]
u_stat, p = mannwhitneyu(group0, group1)
print(f"U Stat = {u_stat:.3f},  p = {p:.4g}")

Spline predictor

In [ ]:
# Create spline-transformed predictor

# 1. Filter rows where n_actors > 0 and not null
df_filtered = df[df["n_actors"].notna() & (df["n_actors"] > 0)].copy()

# 2. Generate the spline basis
spline = dmatrix(
    "bs(n_actors, df=4, degree=3, include_intercept=False)",
    data=df_filtered,
    return_type='dataframe'
)

X = sm.add_constant(spline)
model_act = sm.Logit(df_filtered["reprog"], X).fit()
print(model_act.summary())

In [ ]:
print("AIC:", model_act.aic)

In [ ]:
# Create range of values
x_vals = np.linspace(df["n_actors"].min(), df["n_actors"].max(), 100)
spline_basis = dmatrix("bs(x_vals, df=4, degree=3, include_intercept=False)", {"x_vals": x_vals}, return_type='dataframe')

# Predict probabilities
X_pred = sm.add_constant(spline_basis)
y_pred = model_act.predict(X_pred)

plt.plot(x_vals, y_pred)
plt.xlabel("n_actors")
plt.ylabel("Predicted probability of reprog")
plt.title("Nonlinear effect of n_actors on reprog")
plt.grid(True)
plt.show()

In [ ]:
# Define bins and labels
df["actor_group"] = pd.Series(pd.NA, index=df.index)

bins = [0, 1, 3, 5, 9, np.inf]
labels = ["0–1", "2–3", "4–5", "6–9", "10+"]

bins = [0, 1, 3, 5, 9, np.inf]
labels = ["1", "2–3", "4–5", "6–9", "10+"]
df["actor_group"] = pd.cut(df["n_actors"], bins=bins, labels=labels, right=True)

df["actor_group"] = pd.cut(df["n_actors"], bins=bins, labels=labels)

# Check group sizes
print(df["actor_group"].value_counts(dropna=False))

In [ ]:
group_means = df.groupby("actor_group")["reprog"].mean().reset_index()

sns.barplot(x="actor_group", y="reprog", data=group_means)
plt.ylabel("Reprogramming rate")
plt.title("Reprog rate by actor group")
plt.show()

In [ ]:
# Calculate mean reprog rate per group
group_means = df.groupby("actor_group")["reprog"].mean().reset_index()

# Format and print as percentages
for _, row in group_means.iterrows():
    group = row["actor_group"]
    rate = row["reprog"] * 100  # Convert to percentage
    print(f"Actor group {group}: {rate:.1f}% reprogramming rate")

In [ ]:
# Step 2: One-hot encode (and convert to float to avoid dtype=object)
X = pd.get_dummies(df["actor_group"].dropna(), drop_first=True).astype(float)

# Step 3: Add constant for intercept
X = sm.add_constant(X)

# Step 4: Define y and match indices (in case any NA rows were dropped)
y = df.loc[X.index, "reprog"]

# Step 5: Fit the model
model = sm.Logit(y, X).fit()
print(model.summary())

In [ ]:
print(df.groupby("actor_group")["reprog"].value_counts().unstack(fill_value=0))

In [ ]:
print("Odds ratios:")
print(np.exp(model.params))

In [ ]:
# ── 0. Filter NaN actor_group entries once ─────────────────────────────
df_valid = df[df["actor_group"].notna()].copy()

# ── 1. Observed statistics ─────────────────────────────────────────────
table_obs = pd.crosstab(df_valid["actor_group"], df_valid["reprog"])
chi2, p_chi2, dof, expected = chi2_contingency(table_obs)

v_obs        = cramers_v(table_obs)
v_corr_obs   = cramers_v_bergsma(table_obs)

# ── 2. Bootstrap confidence intervals ──────────────────────────────────
B = 5_000
boot_v        = np.empty(B)
boot_v_corr   = np.empty(B)

for i in range(B):
    sample_df = df_valid.sample(n=len(df_valid), replace=True)
    tbl = pd.crosstab(sample_df["actor_group"], sample_df["reprog"])
    boot_v[i]      = cramers_v(tbl)
    boot_v_corr[i] = cramers_v_bergsma(tbl)

ci_v        = np.percentile(boot_v,        [2.5, 97.5])
ci_v_corr   = np.percentile(boot_v_corr,   [2.5, 97.5])

# ── 3. Permutation tests ───────────────────────────────────────────────
P = 10_000
perm_v        = np.empty(P)
perm_v_corr   = np.empty(P)

reprog_vals = df_valid["reprog"].values.copy()
for i in range(P):
    np.random.shuffle(reprog_vals)
    tbl = pd.crosstab(df_valid["actor_group"], reprog_vals)
    perm_v[i]      = cramers_v(tbl)
    perm_v_corr[i] = cramers_v_bergsma(tbl)

p_perm_v      = (np.sum(perm_v      >= v_obs      ) + 1) / (P + 1)
p_perm_vcorr  = (np.sum(perm_v_corr >= v_corr_obs) + 1) / (P + 1)

# ── 4. Print results ───────────────────────────────────────────────────
print(f"Chi-square statistic      : {chi2:.3f}  (dof={dof}, p={p_chi2:.3f})")

print(f"Cramér’s V : {v_obs:.3f}  "
      f"[95 % CI {ci_v[0]:.3f} – {ci_v[1]:.3f}]  "
      f"Permutation p = {p_perm_v:.3f}")

print(f"Cramér’s V (Bergsma corr.): {v_corr_obs:.3f}  "
      f"[95 % CI {ci_v_corr[0]:.3f} – {ci_v_corr[1]:.3f}]  "
      f"Permutation p = {p_perm_vcorr:.3f}")

In [ ]:
X = df_valid["actor_group"].astype("category").cat.codes
Y = df_valid["reprog"].astype(int)

# Mutual information I(X; Y)
mi = mutual_info_score(X, Y)

# Entropy H(Y)
p_y = np.bincount(Y) / len(Y)
h_y = entropy(p_y, base=2)

# Theil's U
theils_u = mi / h_y if h_y != 0 else 0

print(f"Theil’s U (reprog | actors): {theils_u:.3f}")

#### Duration

In [ ]:
print(df["duration_minutes"].describe())

In [ ]:
df["duration_minutes"].hist(bins=20)
plt.xlabel("Duration")
plt.ylabel("Frequency")
plt.title("Distribution of duration")
plt.show()

In [ ]:
# Filter outliers and shows by duration 
duration_clean = df[(df["duration_minutes"] >= 30) & (df["duration_minutes"] <= 105)].copy()

bins = [29, 45, 60, 70, 85, 110]
labels = ["30–44", "45–59", "60–69", "70–84", "85–105"]

duration_clean["duration_group"] = pd.cut(duration_clean["duration_minutes"], bins=bins, labels=labels)

print(duration_clean["duration_group"].value_counts().sort_index())

In [ ]:
reprog_order = sorted(duration_clean["reprog"].unique())

# Compute median duration for reprog = 0 and reprog = 1
medians = duration_clean.groupby("reprog")["duration_minutes"].median()

# ────────────────────────────────────────────────
# Plot
plt.figure(figsize=(7, 5))
sns.stripplot(
    data=duration_clean,
    x="reprog",
    y="duration_minutes",
    palette=["#1f77b4", "#ff7f0e"],  # blue for 0, orange for 1
    jitter=True,
    order=reprog_order,
    alpha=0.8
)

# Add red horizontal median lines
for i, rep_val in enumerate(reprog_order):
    plt.hlines(
        y=medians[rep_val],
        xmin=i - 0.2, xmax=i + 0.2,
        color="red", linewidth=2
    )

# ────────────────────────────────────────────────
# Style
plt.title("Distribution by Reprogramming")
plt.xlabel("Reprog. in Paris")
plt.ylabel("Duration (min.)")
plt.xticks([0, 1], ["No", "Yes"])
sns.despine()
plt.tight_layout()

# Save to PDF (optional)
file = os.path.join(FIG_PATH, "duration_vs_reprog_bins.pdf")
plt.savefig(file, dpi=300, bbox_inches='tight')
plt.show()

print(f"✅ Saved plot to {file}")

In [ ]:
# Observed statistics
# ────────────────────────────────────────────────────────────────
table_obs = pd.crosstab(duration_clean["duration_group"], duration_clean["reprog"])
chi2, p_chi2, dof, _ = chi2_contingency(table_obs)

v_obs       = cramers_v(table_obs)
v_corr_obs  = cramers_v_bergsma(table_obs)


# Bootstrap 95 % CIs
# ────────────────────────────────────────────────────────────────
B = 5_000
boot_v, boot_v_corr = np.empty(B), np.empty(B)

for i in range(B):
    samp = duration_clean.sample(len(duration_clean), replace=True)
    tbl  = pd.crosstab(samp["duration_group"], samp["reprog"])
    boot_v[i]      = cramers_v(tbl)
    boot_v_corr[i] = cramers_v_bergsma(tbl)

ci_v       = np.percentile(boot_v,       [2.5, 97.5])
ci_v_corr  = np.percentile(boot_v_corr,  [2.5, 97.5])


# Permutation test (P = 10 000)
# ────────────────────────────────────────────────────────────────
P = 10_000
perm_v, perm_v_corr = np.empty(P), np.empty(P)
reprog_vals = duration_clean["reprog"].values.copy()

for i in range(P):
    np.random.shuffle(reprog_vals)
    tbl = pd.crosstab(duration_clean["duration_group"], reprog_vals)
    perm_v[i]      = cramers_v(tbl)
    perm_v_corr[i] = cramers_v_bergsma(tbl)

p_perm_v      = (np.sum(perm_v      >= v_obs     ) + 1) / (P + 1)
p_perm_vcorr  = (np.sum(perm_v_corr >= v_corr_obs) + 1) / (P + 1)

# Print
# ────────────────────────────────────────────────────────────────
print(f"Chi-square statistic      : {chi2:.3f}  (dof={dof}, p={p_chi2:.3f})")
print(f"Cramér’s V  : {v_obs:.3f}  "
      f"[95 % CI {ci_v[0]:.3f} – {ci_v[1]:.3f}]  "
      f"Permutation p = {p_perm_v:.5f}")
print(f"Cramér’s V (Bergsma corr.): {v_corr_obs:.3f}  "
      f"[95 % CI {ci_v_corr[0]:.3f} – {ci_v_corr[1]:.3f}]  "
      f"Permutation p = {p_perm_vcorr:.5f}")

In [ ]:
X = duration_clean["duration_group"].astype("category").cat.codes
Y = duration_clean["reprog"].astype("int")

# Compute mutual information I(X; Y)
mi = mutual_info_score(X, Y)

# Compute entropy H(Y)
p_y = np.bincount(Y) / len(Y)
h_y = entropy(p_y, base=2)

# Compute Theil's U (Y | X)
theils_u = mi / h_y if h_y != 0 else 0

print(f"Theil’s U (reprog | duration groups): {theils_u:.3f}")

In [ ]:
df_auc = duration_clean[["duration_group", "reprog"]].dropna()

# 2. Compute mean reprog rate per group
duration_probs = df_auc.groupby("duration_group")["reprog"].mean()

# 3. Map back to each row
df_auc["duration_score"] = df_auc["duration_group"].map(duration_probs)

# 4. Compute AUC
auc_value = roc_auc_score(df_auc["reprog"], df_auc["duration_score"])

print(f"AUC (duration_group): {auc_value:.3f}")

In [ ]:
# Pearson correlation
r, p = stats.pearsonr(duration_clean["duration_minutes"], duration_clean["reprog"].dropna())

# Display
print(f"Filtered rows used: {len(df_corr)}")
print(f"Point-biserial correlation r  = {r:.3f}")
print(f"p-value                        = {p:.4g}")

In [ ]:
df_corr = duration_clean[["duration_minutes", "reprog"]].dropna()

# Define feature and target
X = df_corr[["duration_minutes"]]  # must be 2D
y = df_corr["reprog"]

# Fit simple logistic model using only this feature
clf = LogisticRegression(solver='liblinear')
clf.fit(X, y)

# Predict probabilities
y_scores = clf.predict_proba(X)[:, 1]

# Compute AUC
auc = roc_auc_score(y, y_scores)

# Display
print(f"Filtered rows used: {len(df_corr)}")
print(f"AUC (1-feature model): {auc:.3f}")

In [ ]:
# Build the spline basis
# 4 degrees of freedom (= 4 basis splines)
spline_basis = dmatrix(
    "bs(duration_minutes, df=4, degree=3, include_intercept=False)",
    data=duration_clean,
    return_type="dataframe"
)

X = sm.add_constant(spline_basis)     # add intercept
y = duration_clean["reprog"]


model_duration= sm.Logit(y, X).fit()
print(model_duration.summary())

In [ ]:
print("AIC:", model_duration.aic)

In [ ]:
# Predict
duration_grid = np.linspace(duration_clean["duration_minutes"].min(),
                            duration_clean["duration_minutes"].max(), 250)

basis_grid = dmatrix(
    "bs(duration_minutes, df=4, degree=3, include_intercept=False)",
    {"duration_minutes": duration_grid},
    return_type="dataframe"
)

X_grid = sm.add_constant(basis_grid)

# Convert to NumPy for correct matrix operations
X_mat = X_grid.to_numpy()
cov_mat = model_duration.cov_params().to_numpy()

# Prediction and 95% CI
pred_prob = model_duration.predict(X_grid)
logit_pred = model_duration.predict(X_grid, which="linear")

se = np.sqrt(np.sum((X_mat @ cov_mat) * X_mat, axis=1))
logit_ci_low = logit_pred - 1.96 * se
logit_ci_high = logit_pred + 1.96 * se
prob_ci_low = 1 / (1 + np.exp(-logit_ci_low))
prob_ci_high = 1 / (1 + np.exp(-logit_ci_high))

# Plot
plt.figure(figsize=(7, 5))

# Main line and CI in orange tones
plt.plot(duration_grid, pred_prob, lw=2, color="#e6550d", label="Predicted probability")
plt.fill_between(duration_grid, prob_ci_low, prob_ci_high,
                 alpha=0.3, color="#fdae6b", label="95 % CI")

plt.xlabel("Duration (minutes)", color="black")
plt.ylabel("P(reprog = 1)", color="black")

plt.grid(False)

ax = plt.gca()
ax.spines['bottom'].set_color("black")
ax.spines['left'].set_color("black")
ax.spines['bottom'].set_linewidth(1.2)
ax.spines['left'].set_linewidth(1.2)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.title("Spline-based Logistic Fit: Duration → Reprog", color="black")
plt.ylim(0, 1)
plt.legend(frameon=False)
plt.tight_layout()
plt.show()


In [ ]:
mfx = model_duration.get_margeff(at="overall", method="dydx")
print("\n📉 Marginal Effects:")
print(mfx.summary())

In [ ]:
# Create plot with histogram under curve
fig, ax1 = plt.subplots(figsize=(8, 5))

# Primary axis: predicted probability curve
ax1.plot(duration_grid, pred_prob, label="Pred. probability", color="#e6550d")
ax1.fill_between(duration_grid, prob_ci_low, prob_ci_high, alpha=0.3, color="#fdae6b", label="95% CI")
ax1.set_xlabel("Duration (min.)", color="black")
ax1.set_ylabel("P(reprog = 1)", color="black")
ax1.tick_params(axis="y", labelcolor="black")
ax1.set_ylim(0, 1)
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.spines['bottom'].set_color("black")
ax1.spines['left'].set_color("black")
ax1.spines['bottom'].set_linewidth(1.2)
ax1.spines['left'].set_linewidth(1.2)
ax1.grid(False)

# Secondary axis: histogram
ax2 = ax1.twinx()
counts, bins, _ = ax2.hist(duration_clean["duration_minutes"], bins=30, color="gray", alpha=0.3)
ax2.set_ylabel("Show count (histogram)", color="gray")
ax2.tick_params(axis="y", labelcolor="gray")
ax2.set_ylim(0, max(counts) * 1.2)
ax2.spines['top'].set_visible(False)
ax2.spines['left'].set_visible(False)

# Final touches
fig.suptitle("Predicted Probability of Reprogramming by Show Duration\nwith Histogram of Observations", color="black")
fig.tight_layout()
ax1.legend(loc="upper left", frameon=False)
png_path = os.path.join(FIG_PATH, "duration_line.png")
pdf_path = os.path.join(FIG_PATH, "duration_line.pdf")
plt.savefig(png_path, dpi=600, bbox_inches="tight", transparent=False)
plt.savefig(pdf_path, dpi=600, bbox_inches="tight", transparent=False)
plt.show()


In [ ]:
# ── 1️⃣ Histogram: duration distribution split by reprog ──────────────
plt.figure(figsize=(7, 5))
bins = np.arange(30, 106, 5)  # 5-minute bins

plt.hist(duration_clean[duration_clean["reprog"] == 0]["duration_minutes"],
         bins=bins, alpha=0.6, label="Not Reprogrammed", color="#3182bd")
plt.hist(duration_clean[duration_clean["reprog"] == 1]["duration_minutes"],
         bins=bins, alpha=0.6, label="Reprogrammed", color="#fdae6b")

plt.xlabel("Duration (minutes)", color="black")
plt.ylabel("Number of Shows", color="black")

for s in ["bottom", "left"]:
    plt.gca().spines[s].set_color("black")
    plt.gca().spines[s].set_linewidth(1.2)
for s in ["top", "right"]:
    plt.gca().spines[s].set_visible(False)

plt.grid(False)
plt.title("Distribution of Show Durations by Reprogramming Status", color="black")
plt.legend(frameon=False)
plt.tight_layout()
plt.show()


# ── 2️⃣ Fitted spline curve with CI (orange style) ──────────────────
plt.figure(figsize=(7, 5))
plt.plot(duration_grid, pred_prob, lw=2, color="#e6550d", label="Predicted probability")
plt.fill_between(duration_grid, prob_ci_low, prob_ci_high, alpha=0.3, color="#fdae6b", label="95% CI")

plt.xlabel("Duration (minutes)", color="black")
plt.ylabel("Probability of Reprogramming", color="black")
plt.ylim(0, 1)

for s in ["bottom", "left"]:
    plt.gca().spines[s].set_color("black")
    plt.gca().spines[s].set_linewidth(1.2)
for s in ["top", "right"]:
    plt.gca().spines[s].set_visible(False)

plt.grid(False)
plt.title("Effect of Duration on Probability of Reprogramming", color="black")
plt.legend(frameon=False)
plt.tight_layout()
plt.show()


In [ ]:
# ── 1. Filter and quantile-bin the data ───────────────────────────────
df_q = df[(df["duration_minutes"] >= 30) & (df["duration_minutes"] <= 105)].copy()
df_q["duration_bin"] = pd.qcut(df_q["duration_minutes"], q=6, duplicates='drop')

# Bin representative values (medians)
bin_medians = df_q.groupby("duration_bin")["duration_minutes"].median()
df_q["duration_group_value"] = df_q["duration_bin"].map(bin_medians)

# ── 2. Spline basis and model fit ─────────────────────────────────────
spline_basis = dmatrix("bs(duration_group_value, df=4, degree=3, include_intercept=False)",
                       data=df_q, return_type="dataframe")
X = sm.add_constant(spline_basis)
y = df_q["reprog"]

model = sm.Logit(y, X).fit(disp=0)

# ── 3. Prediction curve and CI over a grid ────────────────────────────
grid = np.linspace(df_q["duration_group_value"].min(),
                   df_q["duration_group_value"].max(), 300)
grid_basis = dmatrix("bs(duration_group_value, df=4, degree=3, include_intercept=False)",
                     {"duration_group_value": grid}, return_type="dataframe")
X_grid = sm.add_constant(grid_basis)
y_pred = model.predict(X_grid)

# Confidence intervals
cov = model.cov_params()
se = np.sqrt(np.sum((X_grid @ cov) * X_grid, axis=1))
logit_ci_low = model.predict(X_grid, which="linear") - 1.96 * se
logit_ci_high = model.predict(X_grid, which="linear") + 1.96 * se
ci_low = 1 / (1 + np.exp(-logit_ci_low))
ci_high = 1 / (1 + np.exp(-logit_ci_high))

# ── 4. Histogram data (bar plot for each bin) ─────────────────────────
bin_counts = df_q["duration_bin"].value_counts().sort_index()
bar_x = bin_medians.values
bar_heights = bin_counts.values

# ── 5. Plot spline + observed means + histogram bars ──────────────────
fig, ax1 = plt.subplots(figsize=(8, 5))

# Main curve and confidence band
ax1.plot(grid, y_pred, label="Predicted probability", color="C0")
ax1.fill_between(grid, ci_low, ci_high, alpha=0.25, color="C0", label="95% CI")

# Observed means per bin
bin_means = df_q.groupby("duration_bin")["reprog"].mean()
ax1.scatter(bar_x, bin_means.values, color="black", label="Observed bin means")

ax1.set_xlabel("Duration (binned, median of each quantile)")
ax1.set_ylabel("P(reprog = 1)", color="C0")
ax1.set_ylim(0, 1)
ax1.tick_params(axis="y", labelcolor="C0")

# Secondary axis: histogram bars
ax2 = ax1.twinx()
ax2.bar(bar_x, bar_heights, width=3, color="gray", alpha=0.3, label="Bin counts")
ax2.set_ylabel("Show count (per bin)", color="gray")
ax2.tick_params(axis="y", labelcolor="gray")
ax2.set_ylim(0, max(bar_heights) * 1.2)

# Final touches
plt.title("Spline Model of Reprogramming by Duration (with Bin Histogram)")
fig.tight_layout()
ax1.legend(loc="upper left")
plt.show()


##### Group duration by bins

In [ ]:
# ── 2. Define duration bins and labels ─────────────────────────────
bins = [29, 45, 60, 70, 85, 105]
labels = ["30–44", "45–59", "60–69", "70–84", "85–105"]

duration_clean["duration_group"] = pd.cut(duration_clean["duration_minutes"], bins=bins, labels=labels)

# ── 3. Create dummy variables (reference = "30–44") ───────────────
X = pd.get_dummies(duration_clean["duration_group"], drop_first=True).astype(float)
X = sm.add_constant(X)

y = duration_clean["reprog"]

# ── 4. Fit logistic regression ─────────────────────────────────────
model_duration_bins = sm.Logit(y, X).fit()
print(model_duration_bins.summary())

In [ ]:
print("Odds ratios:")
print(np.exp(model_duration_bins.params))

In [ ]:
probs_by_group = duration_clean.groupby("duration_group")["reprog"].mean()
print(probs_by_group)

In [ ]:
# Coefficients and standard errors from the model
coefs_duration_bins = model_duration_bins.params[1:]         # exclude intercept
errors = model_duration_bins.bse[1:]
labels = coefs_duration_bins.index

# Compute odds ratios and CIs
odds_ratios = np.exp(coefs_duration_bins)
ci_low = np.exp(coefs_duration_bins - 1.96 * errors)
ci_high = np.exp(coefs_duration_bins + 1.96 * errors)

# Add the reference group (OR = 1)
labels = ["30–44"] + list(labels)
odds_ratios = [1.0] + list(odds_ratios)
ci_low = [1.0] + list(ci_low)
ci_high = [1.0] + list(ci_high)

# Plot
plt.figure(figsize=(8, 5))
plt.errorbar(labels[1:], odds_ratios[1:], 
             yerr=[np.array(odds_ratios[1:]) - np.array(ci_low[1:]), 
                   np.array(ci_high[1:]) - np.array(odds_ratios[1:])], 
             fmt='o', capsize=5, lw=2, label="Other durations")

# Add reference point manually
plt.plot(labels[0], odds_ratios[0], 'o', color='gray', label="Reference (30–44 min)")

plt.axhline(1, color='gray', linestyle='--')
plt.title("Odds Ratios of Reprogramming by Duration Group")
plt.ylabel("Odds Ratio (compared to 30–44 min)")
plt.xlabel("Duration Group (minutes)")
plt.ylim(bottom=0)
plt.grid(True)
plt.legend(frameon=False)
plt.tight_layout()
plt.show()

#### Genre

In [ ]:
genre_map = {
    # Théâtre / Café-théâtre
    "théâtre": "Théâtre / Café-théâtre", "café-théâtre": "Théâtre / Café-théâtre",
    "comédie": "Théâtre / Café-théâtre", "drame": "Théâtre / Café-théâtre",
    "boulevard": "Théâtre / Café-théâtre", "humour": "Théâtre / Café-théâtre",
    "théâtre sonore": "Théâtre / Café-théâtre",

    # Spectacle musical / Concert
    "théâtre musical": "Spectacle musical / Concert", "spectacle musical": "Spectacle musical / Concert",
    "concert": "Spectacle musical / Concert", "classique": "Spectacle musical / Concert",
    "expo-concert": "Spectacle musical / Concert", "chanson": "Spectacle musical / Concert",

    # Danse / Danse-théâtre
    "danse": "Danse / Danse-théâtre", "danse-théâtre": "Danse / Danse-théâtre",

    # Cirque / Clown
    "cirque": "Cirque / Clown", "clown": "Cirque / Clown",
    "magie": "Cirque / Clown", "théâtre musical/cirque": "Cirque / Clown",

    # Mime / Marionnettes-objets
    "mime": "Mime / Marionnettes-objets",
    "marionnette-objet": "Mime / Marionnettes-objets",
    "marionnette-objet de 7 mois à 4 ans": "Mime / Marionnettes-objets",

    # Conte / Poésie / Lecture
    "lecture": "Conte / Poésie / Lecture", "poésie": "Conte / Poésie / Lecture",
    "conte": "Conte / Poésie / Lecture", "théâtre-poésie": "Conte / Poésie / Lecture",
}


df["genre_group"] = (
    df["genre"].str.lower().str.strip()
      .str.replace(r"\s*/\s*plein\s+air", "", regex=True)
      .map(genre_map)
)

unmapped = df[df["genre_group"].isna()]
if not unmapped.empty:
    print("⚠️ Warning: unmapped genre values detected:\n")
    print(unmapped["genre"].value_counts())

print("✅ Mapped genre groups:", df["genre_group"].unique())

In [ ]:
genre_reprog_counts = (
    df
      .groupby(["genre_group", "reprog"])
      .size()                         # count rows in each combination
      .unstack(fill_value=0)          # turn reprog values into columns
      .rename(columns={0: "reprog_0", 1: "reprog_1"})
      .loc[[
          "Théâtre / Café-théâtre",
          "Spectacle musical / Concert",
          "Danse / Danse-théâtre",
          "Cirque / Clown",
          "Mime / Marionnettes-objets",
          "Conte / Poésie / Lecture"
      ]]
)

print(genre_reprog_counts)

In [ ]:
genre_stats = (
    df.groupby("genre_group")["reprog"]
      .agg(total="count", reprog_1="sum")
      .assign(percent=lambda d: 100 * d["reprog_1"] / d["total"])
      .sort_values("percent", ascending=False)
)

genre_stats["percent"] = genre_stats["percent"].round(1)

print("🎭 Percentage of shows programmed (reprog = 1) by genre group:\n")
display(genre_stats)

In [ ]:
# Test for Genre Grouped
contingency = pd.crosstab(df["genre_group"], df["reprog"]).values
n = contingency.sum()
r, k = contingency.shape

# Chi-square test
chi2, p_val, dof, expected = chi2_contingency(contingency)

# Uncorrected Cramér’s V
phi2 = chi2 / n
v_uncorrected = np.sqrt(phi2 / min(k - 1, r - 1))

# Bergsma-corrected Cramér’s V
phi2_corr = max(0, phi2 - ((k - 1)*(r - 1))/(n - 1))
r_corr = r - ((r - 1)**2)/(n - 1)
k_corr = k - ((k - 1)**2)/(n - 1)
v_corrected = np.sqrt(phi2_corr / min(k_corr - 1, r_corr - 1))

# Bootstrap CI and permutation p-value
n_iter = 5000
boot_v_uncorr = []
boot_v_corr = []
perm_v = []

for _ in range(n_iter):
    # Bootstrap
    sample_idx = np.random.choice(range(contingency.shape[0]), size=contingency.shape[0], replace=True)
    sample = contingency[sample_idx]
    chi2_b, _, _, _ = chi2_contingency(sample)
    n_b = sample.sum()
    phi2_b = chi2_b / n_b
    v_b_uncorr = np.sqrt(phi2_b / min(k - 1, r - 1))
    phi2_b_corr = max(0, phi2_b - ((k - 1)*(r - 1))/(n_b - 1))
    r_b_corr = r - ((r - 1)**2)/(n_b - 1)
    k_b_corr = k - ((k - 1)**2)/(n_b - 1)
    v_b_corr = np.sqrt(phi2_b_corr / min(k_b_corr - 1, r_b_corr - 1))

    boot_v_uncorr.append(v_b_uncorr)
    boot_v_corr.append(v_b_corr)

    # Permutation
    shuffled = df["reprog"].sample(frac=1, replace=False).reset_index(drop=True)
    perm_table = pd.crosstab(df["genre_group"], shuffled).values
    chi2_p, _, _, _ = chi2_contingency(perm_table)
    phi2_p = chi2_p / perm_table.sum()
    v_perm = np.sqrt(phi2_p / min(k - 1, r - 1))
    perm_v.append(v_perm)

# Confidence intervals
ci_uncorr = np.percentile(boot_v_uncorr, [2.5, 97.5])
ci_corr = np.percentile(boot_v_corr, [2.5, 97.5])

# Permutation p-value
perm_p_value = (np.sum(np.array(perm_v) >= v_uncorrected) + 1) / (len(perm_v) + 1)

# Print results
print(f"Chi-square statistic      : {chi2:.3f}  (dof={dof}, p={p_val:.3f})")
print(f"Cramér’s V (uncorrected)  : {v_uncorrected:.3f}  [95 % CI {ci_uncorr[0]:.3f} – {ci_uncorr[1]:.3f}]  Permutation p = {perm_p_value:.2e}")
print(f"Cramér’s V (Bergsma corr.): {v_corrected:.3f}  [95 % CI {ci_corr[0]:.3f} – {ci_corr[1]:.3f}]  Permutation p = {perm_p_value:.2e}")

In [ ]:
X = df["genre_group"].astype("category").cat.codes
Y = df["reprog"].astype("int")

# Mutual information I(X; Y)
mi = mutual_info_score(X, Y)

# Entropy H(Y)
p_y = np.bincount(Y) / len(Y)
h_y = entropy(p_y, base=2)

# Compute Theil's U (Y | X)
theils_u = mi / h_y if h_y != 0 else 0

print(f"Theil’s U (reprog | genre_group): {theils_u:.3f}")

In [ ]:
# 1. Drop missing data
df_auc = df[["genre_group", "reprog"]].dropna()

# 2. Calculate mean reprog rate per genre
probs = df_auc.groupby("genre_group")["reprog"].mean()

# 3. Map the genre-based score back to each row
df_auc["genre_score"] = df_auc["genre_group"].map(probs)

# 4. Compute AUC using genre_score
auc_value = roc_auc_score(df_auc["reprog"], df_auc["genre_score"])

print(f"AUC (genre_group): {auc_value:.3f}")

In [ ]:
# Drop rows with missing values
genre_clean = df.dropna(subset=["genre_group", "reprog"]).copy()
genre_clean = genre_clean[genre_clean["genre_group"] != "Mime / Marionnettes-objets"]

# Create dummy variables from genre_group
X = pd.get_dummies(genre_clean["genre_group"], drop_first=True)

# Add constant (intercept)
X = sm.add_constant(X)

# Target variable
y = genre_clean["reprog"].astype(int)
X = X.astype(float)

# Fit logistic regression model
model_genre_group = sm.Logit(y, X).fit()
print(model_genre_group .summary())

In [ ]:
print("AIC:", model_genre_group.aic)

In [ ]:
print("Odds ratios:")
np.exp(model_genre_group.params)

In [ ]:
# Get predicted probabilities per group
genre_clean["predicted"] = model_genre_group.predict(X)
group_preds = genre_clean.groupby("genre_group")["predicted"].mean().reset_index()

# Plot
sns.barplot(x="genre_group", y="predicted", data=group_preds)
plt.ylabel("Predicted probability of reprog")
plt.xticks(rotation=45)
plt.title("Predicted reprog by genre_group")
plt.tight_layout()
plt.show()

##### Use raw genre categories

In [ ]:
df["genre_no_plein"] = (
    df["genre"].str.lower().str.strip()
      .str.replace(r"\s*/\s*plein\s+air", "", regex=True)
)
df["genre_no_plein"].value_counts()

In [ ]:
# Count total observations per genre
merge_map = {
    "théâtre musical": "musical",
    "spectacle musical": "musical",
    "chanson": "concert",
    "théâtre sonore": "musical",
    "théâtre musical/cirque": "musical",
    "marionnette-objet de 7 mois à 4 ans":"marionnette-objet",
    "théâtre-poésie":"poésie"
}

df["genre_merged"] = df["genre_no_plein"].replace(merge_map)

# Count merged genres
genre_counts = df["genre_merged"].value_counts()

# Filter to valid genres (at least 5 observations)
valid_genres = genre_counts[genre_counts >= 10].index
df_genre_filtered = df[df["genre_merged"].isin(valid_genres)].copy()

print(df_genre_filtered["genre_merged"].value_counts())

In [ ]:
tmp = (
    df_genre_filtered.groupby("genre_merged")["reprog"]
      .agg(total="count", reprog_1="sum")
      .assign(percent=lambda d: 100 * d["reprog_1"] / d["total"])
      .sort_values("total", ascending=False)
)

tmp["percent"] = tmp["percent"].round(1)

pd.set_option("display.max_rows", None)  # see all rows

print("🏛️ Percentage of shows programmed (reprog = 1) by genre :\n")
display(tmp)

In [ ]:
# Build contingency table
contingency = pd.crosstab(df_genre_filtered["genre_merged"], df_genre_filtered["reprog"]).values
n = contingency.sum()
r, k = contingency.shape

# Chi-square test
chi2, p_val, dof, expected = chi2_contingency(contingency)

# Uncorrected Cramér’s V
phi2 = chi2 / n
v_uncorrected = np.sqrt(phi2 / min(k - 1, r - 1))

# Bergsma-corrected Cramér’s V
phi2_corr = max(0, phi2 - ((k - 1)*(r - 1))/(n - 1))
r_corr = r - ((r - 1)**2)/(n - 1)
k_corr = k - ((k - 1)**2)/(n - 1)
v_corrected = np.sqrt(phi2_corr / min(k_corr - 1, r_corr - 1))

# Bootstrap CI and permutation p-value
n_iter = 5000
boot_v_uncorr = []
boot_v_corr = []
perm_v = []

for _ in range(n_iter):
    # Bootstrap
    sample_idx = np.random.choice(range(contingency.shape[0]), size=contingency.shape[0], replace=True)
    sample = contingency[sample_idx]
    chi2_b, _, _, _ = chi2_contingency(sample)
    n_b = sample.sum()
    phi2_b = chi2_b / n_b
    v_b_uncorr = np.sqrt(phi2_b / min(k - 1, r - 1))
    phi2_b_corr = max(0, phi2_b - ((k - 1)*(r - 1))/(n_b - 1))
    r_b_corr = r - ((r - 1)**2)/(n_b - 1)
    k_b_corr = k - ((k - 1)**2)/(n_b - 1)
    v_b_corr = np.sqrt(phi2_b_corr / min(k_b_corr - 1, r_b_corr - 1))

    boot_v_uncorr.append(v_b_uncorr)
    boot_v_corr.append(v_b_corr)

    # Permutation
    shuffled = df_genre_filtered["reprog"].sample(frac=1, replace=False).reset_index(drop=True)
    perm_table = pd.crosstab(df_genre_filtered["genre_merged"], shuffled).values
    chi2_p, _, _, _ = chi2_contingency(perm_table)
    phi2_p = chi2_p / perm_table.sum()
    v_perm = np.sqrt(phi2_p / min(k - 1, r - 1))
    perm_v.append(v_perm)

# Confidence intervals
ci_uncorr = np.percentile(boot_v_uncorr, [2.5, 97.5])
ci_corr = np.percentile(boot_v_corr, [2.5, 97.5])

# Permutation p-value
perm_p_value = (np.sum(perm_v >= v_uncorrected) + 1) / (len(perm_v) + 1)

# Print results
print(f"Chi-square statistic      : {chi2:.3f}  (dof={dof}, p={p_val:.3f})")
print(f"Cramér’s V (uncorrected)  : {v_uncorrected:.3f}  [95 % CI {ci_uncorr[0]:.3f} – {ci_uncorr[1]:.3f}]  Permutation p = {perm_p_value:.5f}")
print(f"Cramér’s V (Bergsma corr.): {v_corrected:.3f}  [95 % CI {ci_corr[0]:.3f} – {ci_corr[1]:.3f}]  Permutation p = {perm_p_value:.5f}")

In [ ]:
X = df_genre_filtered["genre_merged"].astype("category").cat.codes
Y = df_genre_filtered["reprog"].astype("int")

mi = mutual_info_score(X, Y)

# Entropy H(Y)
p_y = np.bincount(Y) / len(Y)
h_y = entropy(p_y, base=2)

# Compute Theil's U (Y | X)
theils_u = mi / h_y if h_y != 0 else 0

print(f"Theil’s U (reprog | genre_group): {theils_u:.3f}")

In [ ]:
df_auc = df_genre_filtered[["genre_merged", "reprog"]].dropna()

# 2. Compute mean reprog rate per genre
genre_probs = df_auc.groupby("genre_merged")["reprog"].mean()

# 3. Map back to each row
df_auc["genre_score"] = df_auc["genre_merged"].map(genre_probs)

# 4. Compute AUC
auc_value = roc_auc_score(df_auc["reprog"], df_auc["genre_score"])

print(f"AUC (genre_merged): {auc_value:.3f}")

In [ ]:
tbl_genre = pd.crosstab(df_genre_filtered["genre_merged"], df_genre_filtered["reprog"])
tbl_genre.columns = ["reprog_0", "reprog_1"]

# ── 2. Filter genres with enough examples ──────────────────────────
mask = (tbl_genre["reprog_0"] >= 5) & (tbl_genre["reprog_1"] >= 2)
valid_genres = tbl_genre[mask].index.tolist()

dropped_genres = tbl_genre[~mask]
print("Dropped genres:\n", dropped_genres)

df_genre_clean = df_genre_filtered[df_genre_filtered["genre_merged"].isin(valid_genres)].copy()

# ── 3. Fit logistic regression ─────────────────────────────────────
X = pd.get_dummies(df_genre_clean["genre_merged"], drop_first=True).astype(float)
X = sm.add_constant(X)
y = df_genre_clean["reprog"].astype(int)

model_genre = sm.Logit(y, X).fit()
print(model_genre.summary())

In [ ]:
print("AIC:", model_genre.aic)
np.exp(model_genre.params)

##### vizualize pred / genre

In [ ]:
pred_prob = model_genre.predict(X)
pred_logit = model_genre.predict(X, which="linear")

# Compute standard errors for confidence intervals
cov = model_genre.cov_params()
se = np.sqrt(np.sum((X @ cov) * X, axis=1))

logit_ci_low = pred_logit - 1.96 * se
logit_ci_high = pred_logit + 1.96 * se
prob_ci_low = 1 / (1 + np.exp(-logit_ci_low))
prob_ci_high = 1 / (1 + np.exp(-logit_ci_high))

# Combine predictions with genre labels
df_preds = df_genre_clean[["genre_merged"]].copy()
df_preds["pred_prob"] = pred_prob
df_preds["ci_low"] = prob_ci_low
df_preds["ci_high"] = prob_ci_high

# Aggregate to one row per genre
grouped = df_preds.groupby("genre_merged", as_index=False).agg({
    "pred_prob": "mean",
    "ci_low": "mean",
    "ci_high": "mean"
})

In [ ]:
# Sort genres by predicted probability
grouped = grouped.sort_values("pred_prob", ascending=True)

plt.figure(figsize=(8, 6))

# Plot points with error bars
plt.errorbar(
    grouped["pred_prob"],
    grouped["genre_merged"],
    xerr=[
        grouped["pred_prob"] - grouped["ci_low"],
        grouped["ci_high"] - grouped["pred_prob"]
    ],
    fmt='o',
    color="darkred",
    ecolor="darkorange",
    elinewidth=1.5,
    capsize=4,
    markersize=6
)

# Axes and labels
plt.xlabel("Pred. of Reprogr.", color="black")
plt.ylabel("Genre", color="black")
plt.title("Estimated Probability of Reprogramming by Genre", color="black")

# Styling
ax = plt.gca()
ax.spines["left"].set_color("black")
ax.spines["bottom"].set_color("black")
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.tick_params(axis='x', colors='black')
ax.tick_params(axis='y', colors='black')

plt.tight_layout()
plt.grid(False)

# Save as PNG and PDF
png_path = os.path.join(FIG_PATH, "genre_pred_new2.png")
pdf_path = os.path.join(FIG_PATH, "genre_pred_new2.pdf")
plt.savefig(png_path, dpi=600, bbox_inches="tight", transparent=False)
plt.savefig(pdf_path, dpi=600, bbox_inches="tight", transparent=False)

plt.show()

In [ ]:
# Filter data for valid genre_group entries
tbl_grouped = pd.crosstab(df["genre_group"], df["reprog"])
keep_mask = (tbl_grouped.sum(axis=1) >= 5) & (tbl_grouped[1] > 2)
valid_genres_grouped = tbl_grouped[keep_mask].index
df_grouped = df[df["genre_group"].isin(valid_genres_grouped)].copy()

# Heatmap (% reprog per genre_group)
heatmap_data = pd.crosstab(df_grouped["genre_group"], df_grouped["reprog"], normalize='index') * 100

plt.figure(figsize=(8, 5))
sns.heatmap(heatmap_data, annot=True, fmt=".1f", cmap="YlOrBr", cbar_kws={'label': '% Reprog'})
plt.title("Reprogramming Rate by Grouped Genre (Heatmap %)")
plt.xlabel("Reprogrammed (0 = No, 1 = Yes)")
plt.ylabel("Genre Group")
plt.tight_layout()
plt.show()

In [ ]:
# Step 2: Heatmap of % reprog per genre_merged
heatmap_data = pd.crosstab(df_genre_clean["genre_merged"], df_genre_clean["reprog"], normalize='index') * 100

plt.figure(figsize=(10, 6))
sns.heatmap(
    heatmap_data,
    annot=True,
    fmt=".1f",
    cmap="YlOrBr",  # You can replace with "YlGnBu" or "PuRd" for different tones
    cbar_kws={'label': '% Rate'}
)
plt.title("Rate by Genre (%)")
plt.xlabel("Reprogrammed in Paris")
plt.ylabel("Genre")
plt.tight_layout()
png_path = os.path.join(FIG_PATH, "heatmap_genre_reprog.png")
pdf_path = os.path.join(FIG_PATH, "heatmap_genre_reprog.pdf")

# Save in high resolution for print (PNG) and as vector (PDF)
plt.savefig(png_path, dpi=600, bbox_inches="tight", transparent=False)
plt.savefig(pdf_path, dpi=600, bbox_inches="tight", transparent=False)

plt.show()

In [ ]:
# True labels (categorical genre), predicted binary outcome
labels_true = df["genre_group"]
labels_pred = df["reprog"]

# Compute MI metrics
raw_mi = mutual_info_score(labels_true, labels_pred)
norm_mi = normalized_mutual_info_score(labels_true, labels_pred)
adj_mi = adjusted_mutual_info_score(labels_true, labels_pred)

print(f"Mutual Information       : {raw_mi:.3f}")
print(f"Normalized MI            : {norm_mi:.3f}")
print(f"Adjusted Mutual Info     : {adj_mi:.3f}")

In [ ]:
# True labels (categorical genre), predicted binary outcome
labels_true = df["genre_merged"]
labels_pred = df["reprog"]

# Compute MI metrics
raw_mi = mutual_info_score(labels_true, labels_pred)
norm_mi = normalized_mutual_info_score(labels_true, labels_pred)
adj_mi = adjusted_mutual_info_score(labels_true, labels_pred)

print(f"Mutual Information       : {raw_mi:.3f}")
print(f"Normalized MI            : {norm_mi:.3f}")
print(f"Adjusted Mutual Info     : {adj_mi:.3f}")

In [ ]:
# True labels (categorical genre), predicted binary outcome
labels_true = df_genre_clean["genre_merged"]
labels_pred = df_genre_clean["reprog"]

# Compute MI metrics
raw_mi = mutual_info_score(labels_true, labels_pred)
norm_mi = normalized_mutual_info_score(labels_true, labels_pred)
adj_mi = adjusted_mutual_info_score(labels_true, labels_pred)

print(f"Mutual Information       : {raw_mi:.3f}")
print(f"Normalized MI            : {norm_mi:.3f}")
print(f"Adjusted Mutual Info     : {adj_mi:.3f}")

In [ ]:
# Encode genre and reprog as integers
X = df_genre_clean["genre_merged"].astype("category").cat.codes
Y = df_genre_clean["reprog"].astype("int")

mi = mutual_info_score(X, Y)

# Entropy H(Y)
p_y = np.bincount(Y) / len(Y)
h_y = entropy(p_y, base=2)

# Compute Theil's U (Y | X)
theils_u = mi / h_y if h_y != 0 else 0

print(f"Theil’s U (reprog | genre): {theils_u:.3f}")

#### Theater Venues

In [ ]:
venues = (
    df.groupby("festival_theater")["reprog"]
      .agg(total="count", reprog_1="sum")
      .assign(percent=lambda d: 100 * d["reprog_1"] / d["total"])
      .sort_values("percent", ascending=False)
)

# 2. Round for cleaner output
venues["percent"] = venues["percent"].round(1)

# 3. Display full table
pd.set_option("display.max_rows", None)  # optional: see all rows
print("🏛️ Percentage of shows programmed (reprog = 1) by venue:\n")
display(venues)

In [ ]:
len(venues)

In [ ]:
len(df)

In [ ]:
unique_venues_count = df["festival_theater"].nunique()
print(f"🎭 Number of unique venues before grouping: {unique_venues_count}")

In [ ]:
# Keep venues with ≥6 total shows (regardless of reprog signal)


tbl_venue = pd.crosstab(df["festival_theater"], df["reprog"])
tbl_venue.columns = ["reprog_0", "reprog_1"]
tbl_venue["total"] = tbl_venue.sum(axis=1)

# Keep venues with ≥6 shows and ≥1 reprog
valid_venues = tbl_venue[tbl_venue["total"] >= 6].index

# Still keep rare but nonzero-signal venues
small_signal_venues = tbl_venue[
    (tbl_venue["total"] < 6)
].index

def group_venue(v):
    if v in valid_venues:
        return v
    else :
        return "Small Venues"

df_venues=df.copy()
df_venues["venue_grouped"] = df_venues["festival_theater"].apply(group_venue)
df_venues = df_venues[df_venues["venue_grouped"].notna()].copy()

In [ ]:
df_venues["venue_grouped"].value_counts()

In [ ]:
len(df_venues)

In [ ]:
print(" Venues grouped under 'Small Venues':")
len(small_signal_venues)

In [ ]:
len(df_venues)

In [ ]:
unique_venues_count = df_venues["venue_grouped"].nunique()
print(f"🎭 Number of unique venues after grouping: {unique_venues_count}")

In [ ]:
# 2. Aggregate reprogramming stats by grouped venue
venues = (
    df_venues.groupby("venue_grouped")["reprog"]
      .agg(total="count", reprog_1="sum")
      .assign(percent=lambda d: 100 * d["reprog_1"] / d["total"])
      .sort_values("percent", ascending=False)
)

# 3. Round for cleaner display
venues["percent"] = venues["percent"].round(1)

# 4. Display full table
pd.set_option("display.max_rows", None)
print("🏛️ Percentage of shows programmed (reprog = 1) by grouped venue:\n")
display(venues)

In [ ]:
# ── 1.  Contingency table (venue vs reprog) ────────────────────────
ct = pd.crosstab(df_venues["venue_grouped"], df_venues["reprog"]).values
n  = ct.sum()
r, k = ct.shape

# ── 2.  Chi-square and Cramér’s V (raw + corrected) ────────────────
chi2, p_chi2, dof, _ = chi2_contingency(ct)

phi2   = chi2 / n
v_raw  = np.sqrt(phi2 / min(k - 1, r - 1))

phi2_c = max(0, phi2 - ((k - 1) * (r - 1)) / (n - 1))      # Bergsma correction
r_corr = r - ((r - 1) ** 2) / (n - 1)
k_corr = k - ((k - 1) ** 2) / (n - 1)
v_corr = np.sqrt(phi2_c / min(k_corr - 1, r_corr - 1))

# ── 3.  Bootstrap CIs  & permutation p-value ───────────────────────
B = 5000
boot_raw, boot_corr, perm_raw = [], [], []

for _ in range(B):
    # bootstrap rows of df
    samp_df = df_venues.sample(len(df_venues), replace=True)
    samp_ct = pd.crosstab(samp_df["venue_grouped"], samp_df["reprog"]).values
    chi2_b, _, _, _ = chi2_contingency(samp_ct)
    phi2_b   = chi2_b / samp_ct.sum()
    boot_raw.append(np.sqrt(phi2_b / min(k - 1, r - 1)))

    phi2_b_c = max(0, phi2_b - ((k - 1)*(r - 1)) / (samp_ct.sum() - 1))
    boot_corr.append(
        np.sqrt(phi2_b_c / min(k_corr - 1, r_corr - 1))
    )

    # permutation
    shuffled = df_venues["reprog"].sample(frac=1, replace=False).reset_index(drop=True)
    perm_ct  = pd.crosstab(df_venues["venue_grouped"], shuffled).values
    chi2_p, _, _, _ = chi2_contingency(perm_ct)
    phi2_p = chi2_p / perm_ct.sum()
    perm_raw.append(np.sqrt(phi2_p / min(k - 1, r - 1)))

# CIs (2.5 % – 97.5 %)
ci_raw  = np.percentile(boot_raw,  [2.5, 97.5])
ci_corr = np.percentile(boot_corr, [2.5, 97.5])

# permutation p-value (+1 correction)
perm_p_raw = (np.sum(np.array(perm_raw) >= v_raw) + 1) / (B + 1)

# ── 4.  Print results ──────────────────────────────────────────────
print(f"χ² = {chi2:.3f}  (dof={dof}, p = {p_chi2:.2e})")
print(f"Cramér’s V (raw)  : {v_raw:.3f}  [95 % CI {ci_raw[0]:.3f} – {ci_raw[1]:.3f}]  "
      f"perm-p = {perm_p_raw:.4f}")
print(f"Cramér’s V (corr) : {v_corr:.3f}  [95 % CI {ci_corr[0]:.3f} – {ci_corr[1]:.3f}]  "
      f"perm-p = {perm_p_raw:.4f}")

In [ ]:
X = df_venues["venue_grouped"].astype("category").cat.codes
Y = df_venues["reprog"].astype("int")

mi = mutual_info_score(X, Y)

# Entropy H(Y)
p_y = np.bincount(Y) / len(Y)
h_y = entropy(p_y, base=2)

# Compute Theil's U (Y | X)
theils_u = mi / h_y if h_y != 0 else 0

print(f"Theil’s U (reprog | genre_group): {theils_u:.3f}")

In [ ]:
from sklearn.metrics import roc_auc_score

# 1. Drop missing data
df_auc = df_venues[["venue_grouped", "reprog"]].dropna()

# 2. Calculate mean reprog rate per venue (acts as a predicted "score")
venue_probs = df_auc.groupby("venue_grouped")["reprog"].mean()

# 3. Map back to the full DataFrame
df_auc["venue_score"] = df_auc["venue_grouped"].map(venue_probs)

# 4. Compute AUC
auc_value = roc_auc_score(df_auc["reprog"], df_auc["venue_score"])

print(f"AUC (venue_grouped): {auc_value:.3f}")

In [ ]:
# Define features and target
tbl_venue = pd.crosstab(df_venues["venue_grouped"], df_venues["reprog"])
tbl_venue.columns = ["reprog_0", "reprog_1"]

# 2. Keep only venues where at least 1 show was reprogrammed
venues_with_reprog = tbl_venue[tbl_venue["reprog_1"] > 0].index

# 3. Filter df_venues
df_pred = df_venues[df_venues["venue_grouped"].isin(venues_with_reprog)].copy()
X = pd.get_dummies(df_pred["venue_grouped"], drop_first=True).astype(float)
X = sm.add_constant(X)
y = df_pred["reprog"].astype(int)

# Fit logistic regression
model_venues = sm.Logit(y, X).fit()
print(model_venues.summary())

In [ ]:
# Print odds ratios
print("AIC:", model_venues.aic)
print("\nOdds ratios:")
print(np.exp(model_venues.params).round(2))

In [ ]:
summary = model_venues.summary2().tables[1]
summary_sorted = summary.sort_values("P>|z|")
summary_sorted.head(10)

In [ ]:
# Filter for statistically significant results
significant = summary[summary["P>|z|"] < 0.05]

# Display
print("Statistically significant theater venues (p < 0.05):")
print(significant)

In [ ]:
predictions = model_venues.get_prediction(X)
summary_frame = predictions.summary_frame(alpha=0.05)
grouped = df_pred["venue_grouped"].to_frame().copy()
grouped["pred_prob"] = summary_frame["predicted"]
grouped["ci_lower"] = summary_frame["ci_lower"]
grouped["ci_upper"] = summary_frame["ci_upper"]

grouped = grouped.groupby("venue_grouped").agg(
    pred_prob=("pred_prob", "mean"),
    ci_low=("ci_lower", "mean"),
    ci_high=("ci_upper", "mean")
).reset_index()

# Sort by predicted probability (optional)
grouped = grouped.sort_values("pred_prob", ascending=True)


plt.figure(figsize=(9, len(grouped) * 0.3))
plt.errorbar(
    grouped["pred_prob"],
    grouped["venue_grouped"],
    xerr=[
        grouped["pred_prob"] - grouped["ci_low"],
        grouped["ci_high"] - grouped["pred_prob"]
    ],
    fmt='o',
    color="darkred",
    ecolor="darkred",
    elinewidth=1.3,
    capsize=3,
    markersize=5
)

# Aesthetic tweaks
plt.xlabel("Predicted Probability of Reprogramming")
plt.ylabel("Venue")
plt.title("Predicted Probability of Reprogramming by Venue (with 95% CI)")
plt.tick_params(axis='both', colors='black')
plt.grid(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_color('black')
plt.gca().spines['bottom'].set_color('black')
plt.tight_layout()
plt.show()

In [ ]:
# Limit to the first 8 venues after sorting
top_venues = grouped.tail(12)

# Plot
plt.figure(figsize=(9, len(top_venues) * 0.5))
plt.errorbar(
    top_venues["pred_prob"],
    top_venues["venue_grouped"],
    xerr=[
        top_venues["pred_prob"] - top_venues["ci_low"],
        top_venues["ci_high"] - top_venues["pred_prob"]
    ],
    fmt='o',
    color="darkred",        # dot color
    ecolor="orange",        # CI bar color
    elinewidth=1.3,
    capsize=3,
    markersize=5
)

# Aesthetic tweaks
plt.xlabel("Pred. Probability of Reprog.")
plt.ylabel("Venue")
plt.title("Top 8 Venues by Predicted Reprogramming Probability (with 95% CI)")
plt.tick_params(axis='both', colors='black')
plt.grid(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_color('black')
plt.gca().spines['bottom'].set_color('black')
plt.tight_layout()

png_path = os.path.join(FIG_PATH, "reprog_venues.png")
pdf_path = os.path.join(FIG_PATH, "reprog_venues.pdf")

plt.savefig(png_path, dpi=300, bbox_inches='tight')  # PNG with high resolution
plt.savefig(pdf_path, bbox_inches='tight')           # PDF (vector quality)

plt.show()



In [ ]:
# AIC and BIC comparison 
print("🟦 Duration-only model:")
print(f"AIC: {model_duration.aic:.2f}")
print(f"BIC: {model_duration.bic:.2f}\n")

print("🟩 Duration-grouped model:")
print(f"AIC: {model_duration_bins.aic:.2f}")
print(f"BIC: {model_duration_bins.bic:.2f}\n")

print("🟪 Actors-only model :")
print(f"AIC: {model_act.aic:.2f}")
print(f"BIC: {model_act.bic:.2f}\n")

print("⬛⬜🟫   Genre-raw model:")
print(f"AIC: {model_genre.aic:.2f}")
print(f"BIC: {model_genre.bic:.2f}\n")

print("🟨 Genre-grouped model:")
print(f"AIC: {model_genre_group.aic:.2f}")
print(f"BIC: {model_genre_group.bic:.2f}\n")

print("🟥 Venues-only model:")
print(f"AIC: {model_venues.aic:.2f}")
print(f"BIC: {model_venues.bic:.2f}\n")


#### Combined Models

##### Duration and venues

In [ ]:
# ── 2. Filter to valid duration range ─────────────────────────────────
df_pred = df_pred[df_pred["venue_grouped"].isin(venues_with_reprog)].copy()

df_pred = df_pred[(df_pred["duration_minutes"] >= 30) & (df_pred["duration_minutes"] <= 105)].copy()

# ── 3. Spline basis for duration ──────────────────────────────────────
spline_basis = dmatrix("bs(duration_minutes, df=4, degree=3, include_intercept=False)",
                       data=df_pred, return_type='dataframe')

venue_dummies = pd.get_dummies(df_pred["venue_grouped"], drop_first=True).astype(float)

# ── 5. Combine features into design matrix ────────────────────────────
X = pd.concat([spline_basis, venue_dummies], axis=1)
X = sm.add_constant(X)
y = df_pred["reprog"].astype(int)

# ── 6. Fit logistic regression ────────────────────────────────────────
model_venues_dur = sm.Logit(y, X).fit()
print(model_venues_dur.summary())

In [ ]:
print("AIC:", model_venues_dur.aic)

In [ ]:
# Extract coefficients and p-values
summary = model_venues_dur.summary2().tables[1]
summary = summary.reset_index().rename(columns={"index": "terms", "Coef.": "coef", "P>|z|": "pval"})

# Filter out intercept
summary = summary[summary["terms"] != "const"]

# Top 10 venues by coefficient
top10 = summary.sort_values("pval", ascending=True).head(15)
print("Top 15 terms by estimated effect:")
top10[["terms", "coef", "pval"]]

##### Duration and raw genre

In [ ]:
print("🔍 Constant or near-constant columns:")
print((X.nunique() == 1).sum(), "columns with constant values")

print("🔍 Columns with only one unique value:")
print(X.loc[:, X.nunique() == 1].columns.tolist())

In [ ]:
df_pred = df[(df["duration_minutes"] >= 30) & (df["duration_minutes"] <= 105)].copy()

# ── 4. Compute genre frequency table ───────────────────────────────────
tbl_genre = pd.crosstab(df_pred["genre_merged"], df_pred["reprog"])
tbl_genre.columns = ["reprog_0", "reprog_1"]

mask = (tbl_genre["reprog_0"] >= 5) & (tbl_genre["reprog_1"] >= 1) 
valid_genres = tbl_genre[mask].index
df_pred = df_pred[df_pred["genre_merged"].isin(valid_genres)].copy()

# ── 7. Create spline basis for duration ────────────────────────────────
spline_duration = dmatrix(
    "bs(duration_minutes, df=4, degree=3, include_intercept=False)",
    data=df_pred,
    return_type='dataframe'
)

# ── 8. Create dummy variables for genre ────────────────────────────────
genre_dummies = pd.get_dummies(df_pred["genre_merged"], drop_first=True).astype(float)

# ── 9. Combine predictors ──────────────────────────────────────────────
X = pd.concat([spline_duration.reset_index(drop=True), genre_dummies.reset_index(drop=True)], axis=1)
X = sm.add_constant(X)
y = df_pred["reprog"].astype(int).reset_index(drop=True)

# ── 10. Fit logistic regression ────────────────────────────────────────
model_genre_duration = sm.Logit(y, X).fit(method='lbfgs', maxiter=200)
print(model_genre_duration.summary())


In [ ]:
print("AIC:", model_genre_duration.aic)

In [ ]:
# Extract coefficients and p-values
summary = model_genre_duration.summary2().tables[1]
summary = summary.reset_index().rename(columns={"index": "term", "Coef.": "coef", "P>|z|": "pval"})

# Exclude intercept
summary = summary[summary["term"] != "const"]

# Top 10 terms by lowest p-value
top10 = summary.sort_values("pval", ascending=True).head(15)
print("Top 15 terms by statistical significance (lowest p-values):")
top10[["term", "coef", "pval"]]

In [ ]:
##### Venues and raw genre

In [ ]:
df_pred = df_venues[df_venues["venue_grouped"].isin(venues_with_reprog)].copy()

tbl_genre = pd.crosstab(df_pred["genre_merged"], df_pred["reprog"])
tbl_genre.columns = ["reprog_0", "reprog_1"]

mask = (tbl_genre["reprog_0"] >= 5) & (tbl_genre["reprog_1"] >= 1) 
valid_genres = tbl_genre[mask].index
df_pred = df_pred[df_pred["genre_merged"].isin(valid_genres)]

# ── 5. Dummy encode genre and venue ─────────────────────────────────────
genre_dummies = pd.get_dummies(df_pred["genre_merged"], drop_first=True).astype(float)
venue_dummies = pd.get_dummies(df_pred["venue_grouped"], drop_first=True).astype(float)

# ── 6. Combine all predictors ───────────────────────────────────────────
X = pd.concat([genre_dummies, venue_dummies], axis=1)
X = sm.add_constant(X)
y = df_pred["reprog"].astype(int)

# ── 7. Fit logistic regression ──────────────────────────────────────────
model_venues_genre = sm.Logit(y, X).fit(method='lbfgs', maxiter=200)
print(model_venues_genre.summary())


In [ ]:
print("AIC:", model_venues_genre.aic)

In [ ]:
# Extract coefficients and p-values
summary = model_venues_genre.summary2().tables[1]
summary = summary.reset_index().rename(columns={"index": "term", "Coef.": "coef", "P>|z|": "pval"})

# Exclude intercept
summary = summary[summary["term"] != "const"]

# Top 10 terms by lowest p-value
top10 = summary.sort_values("pval", ascending=True).head(15)
print("Top 15 terms by statistical significance (lowest p-values):")
top10[["term", "coef", "pval"]]

In [ ]:
##### Venues, raw genre, duration

In [ ]:
# ── 1. Filter venue, genre, and duration ───────────────────────────────
df_pred = df_venues[df_venues["venue_grouped"].isin(venues_with_reprog)].copy()

tbl_genre = pd.crosstab(df_pred["genre_merged"], df_pred["reprog"])
tbl_genre.columns = ["reprog_0", "reprog_1"]
valid_genres = tbl_genre[(tbl_genre["reprog_0"] >= 5) & (tbl_genre["reprog_1"] >= 1)].index

df_pred = df_pred[df_pred["genre_merged"].isin(valid_genres)]
df_pred = df_pred[(df_pred["duration_minutes"] >= 30) & (df_pred["duration_minutes"] <= 105)].copy()

# ── 2. Feature creation ────────────────────────────────────────────────
spline_duration = dmatrix(
    "bs(duration_minutes, df=4, degree=3, include_intercept=False)",
    data=df_pred,
    return_type='dataframe'
)

genre_dummies = pd.get_dummies(df_pred["genre_merged"], drop_first=True).astype(float)
venue_dummies = pd.get_dummies(df_pred["venue_grouped"], drop_first=True).astype(float)

# ── 3. Combine all predictors ──────────────────────────────────────────
X = pd.concat([spline_duration, genre_dummies, venue_dummies], axis=1)
X = sm.add_constant(X)
y = df_pred["reprog"].astype(int)

# ── 4. Fit logistic regression ─────────────────────────────────────────
model_all = sm.Logit(y, X).fit(method='lbfgs', maxiter=200)
print(model_all.summary())

In [ ]:
print("AIC:", model_all.aic)

In [ ]:
# Extract coefficients and p-values
summary = model_all.summary2().tables[1]
summary = summary.reset_index().rename(columns={"index": "term", "Coef.": "coef", "P>|z|": "pval"})

# Exclude intercept
summary = summary[summary["term"] != "const"]

# Top 10 terms by lowest p-value
top10 = summary.sort_values("pval", ascending=True).head(20)
print("Top 20 terms by statistical significance (lowest p-values):")
top10[["term", "coef", "pval"]]

In [ ]:
top_predictors = (
    model_all.summary2().tables[1]
    .reset_index()
    .rename(columns={"index": "feature"})
    .sort_values(by="Coef.", key=abs, ascending=False))

# Filter for statistically significant predictors (p < 0.05)
significant_predictors = top_predictors[top_predictors["P>|z|"] < 0.05]

# Display top 10 significant predictors
significant_predictors.head(10)

In [ ]:
X_vif = X.drop(columns='const', errors='ignore')  # remove constant if present

# 2. Calculate VIFs
vif_data = pd.DataFrame()
vif_data["feature"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]

# 3. Sort and display
vif_data = vif_data.sort_values("VIF", ascending=False)
vif_data.head(10)

In [ ]:
odds_ratios = np.exp(model_all.params)
neg_odds = odds_ratios.sort_values().head(10)
print("\n🔻 Top 10 negative odds ratios:")
print(neg_odds)

# ── 6. Marginal effects (Average Marginal Effects) ───────────────────
mfx = model_all.get_margeff(at="overall", method="dydx")
print("\n📉 Marginal Effects:")
print(mfx.summary())

In [ ]:
# Get predicted probabilities with confidence intervals
pred = model_all.get_prediction(X)
summary_frame = pred.summary_frame(alpha=0.05)

df_pred["pred_prob"] = summary_frame["predicted"]
df_pred["ci_lower"] = summary_frame["ci_lower"]
df_pred["ci_upper"] = summary_frame["ci_upper"]

# Aggregate by venue
venue_avg = (
    df_pred.groupby("venue_grouped")[["pred_prob", "ci_lower", "ci_upper"]]
    .mean()
    .sort_values("pred_prob", ascending=True)
)

# Plot
plt.figure(figsize=(9, len(venue_avg) * 0.3))
plt.errorbar(
    venue_avg["pred_prob"],
    venue_avg.index,
    xerr=[
        venue_avg["pred_prob"] - venue_avg["ci_lower"],
        venue_avg["ci_upper"] - venue_avg["pred_prob"]
    ],
    fmt='o',
    color="darkred",
    ecolor="orange",
    capsize=3
)
plt.xlabel("Predicted Probability of Reprogramming")
plt.ylabel("Venue")
plt.title("Predicted Reprogramming Probabilities by Venue")
plt.tight_layout()
plt.show()


In [ ]:
# ── 1. Predict probabilities with confidence intervals ──────────────
pred = model_all.get_prediction(X)
summary_frame = pred.summary_frame()

df_pred["pred_prob"] = summary_frame["predicted"]
df_pred["ci_lower"] = summary_frame["ci_lower"]
df_pred["ci_upper"] = summary_frame["ci_upper"]

# ── 2. Aggregate by venue ───────────────────────────────────────────
venue_avg = (
    df_pred.groupby("venue_grouped")[["pred_prob", "ci_lower", "ci_upper"]]
    .mean()
    .sort_values("pred_prob", ascending=True)
    .tail(15)  # ⬅️ Top 15 venues only
)

# ── 3. Plot with error bars ─────────────────────────────────────────
plt.figure(figsize=(8, 5))
plt.errorbar(
    venue_avg["pred_prob"],
    venue_avg.index,
    xerr=[
        venue_avg["pred_prob"] - venue_avg["ci_lower"],
        venue_avg["ci_upper"] - venue_avg["pred_prob"]
    ],
    fmt='o',
    color="darkred",
    ecolor="orange",
    capsize=3,
    markersize=5,
    elinewidth=1.2
)

# ── 4. Customize appearance ─────────────────────────────────────────
plt.xlabel("Predicted Probability of Reprogramming")
plt.ylabel("Venue")
plt.title("Top 15 Venues by Predicted Reprogramming Probability (95% CI)")
plt.tick_params(axis='both', labelsize=8)
plt.grid(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.tight_layout()

# ── 5. Save the figure ──────────────────────────────────────────────
plt.savefig(os.path.join(FIG_PATH, "predicted_venues_top15.png"), dpi=300, bbox_inches="tight")
plt.savefig(os.path.join(FIG_PATH, "predicted_venues_top15.pdf"), bbox_inches="tight")

plt.show()
plt.close()


In [ ]:
sns.scatterplot(data=df_pred, x="duration_minutes", y="pred_prob", hue="genre_merged", alpha=0.5)
plt.title("Predicted Probability vs Duration (colored by Genre)")
plt.ylabel("Predicted Probability of Reprogramming")
plt.xlabel("Duration (minutes)")
plt.tight_layout()
plt.show()

In [ ]:
# 1. Create and fit pipeline

from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

ridge_pipeline = make_pipeline(
    StandardScaler(with_mean=False),
    LogisticRegressionCV(
        penalty="l2",
        solver="liblinear",
        cv=3,
        scoring="neg_log_loss",
        max_iter=1000,
        n_jobs=-1,
        random_state=42
    )
)

# ✅ Fit the model
ridge_pipeline.fit(X, y)  # make sure X is a DataFrame and y is your target

# 2. Access the model
ridge_model = ridge_pipeline.named_steps["logisticregressioncv"]

# ✅ Ensure model is fitted
assert hasattr(ridge_model, "coef_"), "Model must be fitted before accessing .coef_"

# 3. Extract and inspect coefficients
feature_names = X.columns
coefs = pd.Series(ridge_model.coef_[0], index=feature_names)

coefs_sorted = coefs.sort_values(key=abs, ascending=False)

print("🔝 Top predictors (Ridge):")
print(coefs_sorted.head(10))

print("\n🔻 Least influential (Ridge):")
print(coefs_sorted.tail(10))

scoring = {
    "neg_log_loss": "neg_log_loss",
    "accuracy": "accuracy",
    "roc_auc": "roc_auc"
}

results = cross_validate(
    ridge_pipeline, X, y,
    cv=cv,
    scoring=scoring,
    return_train_score=False
)


print("Cross-validated performance (Ridge, full model):")
for metric, values in results.items():
    if "test" in metric:
        print(f"{metric}: {values.mean():.4f}")


In [ ]:
ridge_pipeline.fit(X, y)
best_C = ridge_pipeline.named_steps['logisticregressioncv'].C_[0]
print(f"\nBest regularization (C): {best_C:.5f}")

In [ ]:
# Dummy model that always predicts the majority class
dummy = DummyClassifier(strategy='most_frequent')
dummy.fit(X, y)

# Predict on the same set (or cross-validate)
y_pred = dummy.predict(X)
y_proba = dummy.predict_proba(X)[:, 1]

# Evaluate
print("Baseline Accuracy:", (y_pred == y).mean())
print("Baseline AUC:", roc_auc_score(y, y_proba))
print("Baseline Log loss:", log_loss(y, y_proba))

#### Test XGBoost

In [ ]:
spline_duration = dmatrix(
    "bs(duration_minutes, df=4, degree=3, include_intercept=False)",
    data=df_pred,
    return_type="dataframe"
)
spline_duration.columns = [f"spline_{i}" for i in range(spline_duration.shape[1])]

# 2. Combine all features
X_final = pd.concat([spline_duration, genre_dummies, venue_dummies], axis=1).astype(float)
y_final = df_pred["reprog"].astype(int)

In [ ]:
scale_pos_weight = 2.1 # based on class imbalance (adjust if needed)

# ── 1. Fit XGBoost on filtered data ───────────────────────────────
model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    n_estimators=100,
    max_depth=4
)
model.fit(X_final, y_final)

# ── 2. Predict and evaluate ────────────────────────────────────────
y_pred = model.predict(X_final)

accuracy = accuracy_score(y_final, y_pred)
recall_macro = recall_score(y_final, y_pred, average="macro")
f1_macro = f1_score(y_final, y_pred, average="macro")

print(f"\nAccuracy: {accuracy:.4f}")
print(f"Macro Recall: {recall_macro:.4f}")
print(f"Macro F1-score: {f1_macro:.4f}")

print("\nDetailed Classification Report:")
print(classification_report(y_final, y_pred, target_names=["Not Reprog", "Reprog"]))

# ── 3. Confusion matrix ────────────────────────────────────────────
cm = confusion_matrix(y_final, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Not Reprog", "Reprog"])
disp.plot(cmap="Blues")
plt.title("Confusion Matrix (XGBoost, Filtered Data)")
plt.tight_layout()
plt.show()


In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scoring = {
    "accuracy": "accuracy",
    "roc_auc": "roc_auc",
    "neg_log_loss": "neg_log_loss"
}

# ── 2. Run cross-validation ───────────────────────────────────────────
results = cross_validate(model, X_final, y_final, cv=cv, scoring=scoring, return_estimator=True)

print("\nXGBoost Cross-Validated Performance (scale_pos_weight=2.0):")
for metric in scoring:
    mean_score = results[f'test_{metric}'].mean()
    print(f"{metric}: {mean_score:.4f}")

# ── 3. Feature importance from best estimator ─────────────────────────
# Use the estimator with best ROC AUC
best_idx = np.argmax(results['test_roc_auc'])
best_model = results['estimator'][best_idx]

# Plot feature importances
importances = best_model.feature_importances_
feature_names = X_final.columns

importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False).head(20)

In [ ]:
# ✅ 1. Define and register custom colormap
matplotlib_cmap = LinearSegmentedColormap.from_list(
    "custom_orange_yellow_red", ["yellow", "orange", "darkred"]
)

# ✅ 2. Compute SHAP values
explainer = shap.TreeExplainer(model)
shap_values = explainer(X_final)  # NOT .shap_values()

# ✅ 3. Get top 15 features by mean absolute SHAP value
shap_importance = pd.DataFrame({
    'feature': X_final.columns,
    'mean_abs_shap': np.abs(shap_values.values).mean(axis=0)
}).sort_values(by='mean_abs_shap', ascending=False)

top15_features = shap_importance['feature'].head(15).tolist()
X_top15 = X_final[top15_features]
shap_values_top15 = shap_values[:, [X_final.columns.get_loc(f) for f in top15_features]]

# ✅ 4. Plot SHAP summary for top 15 with custom styling
shap.summary_plot(
    shap_values_top15,
    X_top15,
    plot_type="dot",
    cmap=matplotlib_cmap,
    axis_color="#000000",
    show=False,
    plot_size=(6, 5)
)

# ✅ 5. Customize labels and appearance
plt.xlabel("Impact on Reprogr. Pred.", fontsize=9)
plt.ylabel("Feature", fontsize=9)
plt.tick_params(labelsize=7)
plt.title("Top 15 Predictors (SHAP)", fontsize=9)

# ✅ 6. Save
fig = plt.gcf()
fig.tight_layout()
fig.savefig(os.path.join(FIG_PATH, "shap_summary2.png"), dpi=300, bbox_inches='tight')
fig.savefig(os.path.join(FIG_PATH, "shap_summary2.pdf"), bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
import matplotlib.pyplot as plt

# 1. Split data
X_train, X_val, y_train, y_val = train_test_split(
    X_final, y_final, stratify=y_final, test_size=0.2, random_state=42
)

# 2. Define model
model = xgb.XGBClassifier(
    objective='binary:logistic',
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    n_estimators=100,
    max_depth=4
)

# 3. Set eval_metric if .fit() doesn't accept it directly
model.set_params(eval_metric='logloss')

# 4. Fit with validation set
model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    verbose=False
)

# 5. Extract learning curve results
results = model.evals_result()

# 6. Plot learning curve
epochs = len(results['validation_0']['logloss'])
x_axis = range(epochs)

plt.figure(figsize=(8, 5))
plt.plot(x_axis, results['validation_0']['logloss'], label='Train')
plt.plot(x_axis, results['validation_1']['logloss'], label='Validation')
plt.xlabel("Boosting Round")
plt.ylabel("Log Loss")
plt.title("XGBoost Learning Curve (Log Loss)")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score

precision, recall, _ = precision_recall_curve(y_final, y_proba)
ap = average_precision_score(y_final, y_proba)

plt.plot(recall, precision, label=f"AP = {ap:.2f}")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.legend()
plt.show()


In [ ]:
from sklearn.calibration import calibration_curve

prob_true, prob_pred = calibration_curve(y_final, y_proba, n_bins=10)
plt.plot(prob_pred, prob_true, marker='o', label="Model")
plt.plot([0, 1], [0, 1], 'k--', label="Perfectly Calibrated")
plt.xlabel("Mean Predicted Probability")
plt.ylabel("Fraction of Positives")
plt.title("Calibration Curve")
plt.legend()
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc

y_proba = model.predict_proba(X_final)[:, 1]
fpr, tpr, _ = roc_curve(y_final, y_proba)
roc_auc = auc(fpr, tpr)

plt.plot(fpr, tpr, label=f"ROC curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], 'k--')  # diagonal line
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate (Recall)")
plt.title("ROC Curve")
plt.legend()
plt.show()


#### Time Slot

In [ ]:
def time_str_to_hour(t):
    """Convert '11h15' or '15h' to fractional hour (e.g., 11.25 or 15.0)."""
    if pd.isna(t):
        return np.nan
    try:
        # Remove whitespace and standardize format
        t = str(t).strip().lower()
        if "h" not in t:
            return np.nan
        hour_min = t.split("h")
        hour = int(hour_min[0])
        minute = int(hour_min[1]) if len(hour_min) > 1 and hour_min[1].isdigit() else 0
        return hour + minute / 60
    except Exception:
        return np.nan


df["start_hour"] = df["start_time"].apply(time_str_to_hour)

In [ ]:
df["start_hour"].value_counts()

In [ ]:
# Keep shows that start between 8 AM and midnight
df_time = df[(df["start_hour"] >= 9) & (df["start_hour"] <= 24)].copy()

In [ ]:
spline_time = dmatrix("bs(start_hour, df=4, degree=3, include_intercept=False)",
                      data=df_time, return_type='dataframe')

X = pd.concat([spline_time], axis=1)
X = sm.add_constant(X)
y = df_time["reprog"].astype(int)

model_time = sm.Logit(y, X).fit()
print(model_time.summary())

In [ ]:
print("AIC:", model_time.aic)

In [ ]:
# ── 1. Smooth hour grid ────────────────────────────────────────────────
hour_grid = np.linspace(6, 23, 200)            # adjust min/max if needed

# ── 2. Spline basis for the grid ───────────────────────────────────────
spline_grid = dmatrix(
    "bs(hour_grid, df=4, degree=3, include_intercept=False)",
    {"hour_grid": hour_grid},
    return_type="dataframe"
)

X_grid = sm.add_constant(spline_grid)
logit_pred = model_time.predict(X_grid, which="linear")

# ── 3. Delta-method 95 % CI on the probability scale ───────────────────
cov = model_time.cov_params()
# Ensure design matrix is NumPy array
X_grid_np = X_grid.values  # convert to NumPy array
cov_np = model_time.cov_params().values  # also convert covariance to NumPy

# Compute standard errors on the logit scale
se = np.sqrt(np.einsum("ij,jk,ik->i", X_grid_np, cov_np, X_grid_np))

logit_low  = logit_pred - 1.96 * se
logit_high = logit_pred + 1.96 * se
prob_pred  = 1 / (1 + np.exp(-logit_pred))
prob_low   = 1 / (1 + np.exp(-logit_low))
prob_high  = 1 / (1 + np.exp(-logit_high))

# ── 4. Plot with orange styling & black axes ───────────────────────────
plt.figure(figsize=(8, 5))
plt.plot(hour_grid, prob_pred, color="#e6550d", linewidth=2)
plt.fill_between(hour_grid, prob_low, prob_high, color="#fdae6b", alpha=0.35)

# Axis labels & aesthetics
plt.xlabel("Start Hour of Performance", color="black")
plt.ylabel("Predicted P(reprog = 1)", color="black")
plt.title("Spline Fit: Start Time → Reprogramming Probability", color="black")
plt.ylim(0, 1)

ax = plt.gca()
for spine in ["left", "bottom"]:
    ax.spines[spine].set_color("black")
    ax.spines[spine].set_linewidth(1.2)
for spine in ["top", "right"]:
    ax.spines[spine].set_visible(False)

ax.tick_params(axis="x", colors="black")
ax.tick_params(axis="y", colors="black")
plt.grid(False)
plt.tight_layout()
plt.show()

In [ ]:
# Drop NaNs from both columns
df_valid = df[["start_hour", "reprog"]].dropna()

# Bin start_hour
bins = pd.cut(df_valid["start_hour"], bins=range(0, 25), right=False)

# Compute Mutual Information
mi = mutual_info_score(bins, df_valid["reprog"])
print(f"Mutual Information (start_hour vs reprog): {mi:.3f}")

In [ ]:
df_corr = df[["start_hour", "reprog"]].dropna()

# Pearson correlation (equivalent to point-biserial when one var is binary)
r, p = stats.pearsonr(df_corr["start_hour"], df_corr["reprog"])

# Display
print(f"Filtered rows used: {len(df_corr)}")
print(f"Point-biserial correlation r  = {r:.3f}")
print(f"p-value                        = {p:.4g}")

In [ ]:
df_corr = df[["start_hour", "reprog"]].dropna()

# Define feature and target
X = df_corr[["start_hour"]]  # must be 2D
y = df_corr["reprog"]

# Fit simple logistic model using only this feature
clf = LogisticRegression(solver='liblinear')
clf.fit(X, y)

# Predict probabilities
y_scores = clf.predict_proba(X)[:, 1]

# Compute AUC
auc = roc_auc_score(y, y_scores)

# Display
print(f"Filtered rows used: {len(df_corr)}")
print(f"AUC (1-feature model): {auc:.3f}")

In [ ]:
from rapidfuzz import process, fuzz
import unicodedata

# 1. Extract all unique institution names
unique_insts = exploded["institutions"].dropna().unique().tolist()

# 2. Normalize: lowercase and strip accents
def normalize(text):
    if pd.isna(text): return ""
    text = text.lower().strip()
    return ''.join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn')

normalized_map = {inst: normalize(inst) for inst in unique_insts}

# 3. Group similar names based on fuzzy score
threshold = 80  # adjust as needed (90 = very close match)
canonical_names = {}
used = set()

for inst, norm_inst in normalized_map.items():
    if inst in used:
        continue
    # Find close matches
    matches = process.extract(
        norm_inst,
        [normalize(i) for i in unique_insts],
        scorer=fuzz.token_sort_ratio,
        score_cutoff=threshold,
        limit=None  # return all above threshold
    )
    group = [match[0] for match in matches]
    for match in group:
        # Map all matches to this canonical name
        original = [k for k, v in normalized_map.items() if v == match]
        for orig in original:
            canonical_names[orig] = inst
            used.add(orig)

# 4. Apply to exploded DataFrame
exploded["institution_grouped"] = exploded["institutions"].map(canonical_names).fillna(exploded["institutions"])

In [ ]:
exploded["institution_grouped"].nunique(dropna=True)

In [ ]:
exploded["institutions"].nunique(dropna=True)

In [ ]:
grouped_stats = (
    exploded.groupby("institution_grouped")["reprog"]
            .agg(total_shows="count", reprog_count="sum")
            .assign(percent_reprog=lambda d: (100 * d["reprog_count"] / d["total_shows"]).round(1))
            .sort_values("total_shows", ascending=False)
)

pd.set_option("display.max_rows", None)
display(grouped_stats)

In [ ]:
exploded_grp = exploded.dropna(subset=["institution_grouped", "reprog"])

exploded_grp = exploded_grp.assign(flag=1).reset_index(drop=True)

dummy_grp = exploded_grp.pivot_table(
    index=exploded_grp.index,
    columns="institution_grouped",
    values="flag",
    fill_value=0
)

# attach reprog
dummy_grp["reprog"] = exploded_grp["reprog"].astype(int)

# ------------------------------------------------------------------
# 2.  Chi-square & Cramér’s V for each grouped institution
# ------------------------------------------------------------------
results = []

for col in dummy_grp.columns.drop("reprog"):
    if dummy_grp[col].sum() < 3:          # keep only institutions in ≥5 shows
        continue
    ct = pd.crosstab(dummy_grp[col], dummy_grp["reprog"])
    if ct.shape != (2, 2):
        continue                         # need both 0/1 rows and columns
    chi2, p, *_ = chi2_contingency(ct)
    n  = ct.to_numpy().sum()
    V  = np.sqrt(chi2 / n)               # k-1 = 1 for a 2×2 table
    results.append((col, dummy_grp[col].sum(), V, p))

# ------------------------------------------------------------------
# 3.  Assemble results and display
# ------------------------------------------------------------------
res_df = (pd.DataFrame(results,
                       columns=["institution_grouped",
                                "shows_with_inst",
                                "cramers_v",
                                "p_value"])
          .sort_values("cramers_v", ascending=False)
          .reset_index(drop=True))

pd.set_option("display.max_rows", None)   # show all if desired
print("🏆 Grouped institutions ranked by association strength with reprog:")
display(res_df.head(25))                  # top 25 (adjust as needed)

In [ ]:
import pandas as pd

# 1. Explode the 'institutions' list
exploded = df[df["institutions"].str.len() > 0].explode("institutions")

# 2. Drop rows with missing values (safety)
exploded = exploded.dropna(subset=["institutions", "reprog"])

# 3. Group and calculate stats
inst_stats = (
    exploded.groupby("institutions")["reprog"]
            .agg(total_shows="count", reprog_count="sum")
            .assign(percent_reprog=lambda d: (100 * d["reprog_count"] / d["total_shows"]).round(1))
            .sort_values("total_shows", ascending=False)
)

# 4. Ensure full display (no truncation)
pd.set_option("display.max_rows", None)

# 5. Display result
print("📊 Reprogramming percentage per institution (all listed):")
display(inst_stats)

In [ ]:
len(exploded["institutions"])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 1. Clean and group
df["n_actors"] = pd.to_numeric(df["n_actors"], errors="coerce")
df_filtered = df.dropna(subset=["n_actors", "reprog"])

# 2. Count total and reprogrammed
actor_counts = (
    df_filtered.groupby("n_actors")["reprog"]
    .agg(total="count", reprog="sum")
    .reset_index()
    .astype({"n_actors": int})
)

# 3. Compute non-reprogrammed counts
actor_counts["non_reprog"] = actor_counts["total"] - actor_counts["reprog"]

# 4. Plot stacked bars
plt.figure(figsize=(12, 6))
x = actor_counts["n_actors"]

plt.bar(x, actor_counts["non_reprog"], label="Non-Reprogrammed", color="lightgray")
plt.bar(x, actor_counts["reprog"], bottom=actor_counts["non_reprog"], label="Reprogrammed", color="tomato")

# 5. Style
plt.xlabel("Number of Actors")
plt.ylabel("Number of Shows")
plt.title("🎭 Stacked Distribution of Shows by Number of Actors and Reprog Status")
plt.xticks(x)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 1. Clean and group
df["n_actors"] = pd.to_numeric(df["n_actors"], errors="coerce")
df_filtered = df.dropna(subset=["n_actors", "reprog"])

# 2. Count total shows and reprogrammed shows
actor_counts = (
    df_filtered.groupby("n_actors")["reprog"]
    .agg(total="count", reprog="sum")
    .reset_index()
    .astype({"n_actors": int})
)

# 3. Plot grouped bars
plt.figure(figsize=(12, 6))
bar_width = 0.4
x = actor_counts["n_actors"]

plt.bar(x - bar_width/2, actor_counts["total"], width=bar_width, label="Total shows", color="lightgray")
plt.bar(x + bar_width/2, actor_counts["reprog"], width=bar_width, label="Reprogrammed shows", color="tomato")

# 4. Style
plt.xlabel("Number of Actors")
plt.ylabel("Number of Shows")
plt.title("🎭 Total and Reprogrammed Shows by Number of Actors")
plt.xticks(x)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 1. Clean data
df_viz = df.dropna(subset=["genre_clean", "reprog"])

# 2. Create counts per genre and reprog status
genre_counts = (
    df_viz.groupby(["genre_clean", "reprog"])
    .size()
    .unstack(level=1, fill_value=0)
)

# 3. Ensure both columns 0 and 1 exist
for col in [0, 1]:
    if col not in genre_counts.columns:
        genre_counts[col] = 0

# 4. Normalize to percentages
genre_props = genre_counts.div(genre_counts.sum(axis=1), axis=0) * 100

# 5. Sort by reprog = 1
genre_props = genre_props.sort_values(by=1, ascending=False)

# 6. Plot
plt.figure(figsize=(10, 6))
genre_props[[0, 1]].plot(
    kind="bar",
    stacked=True,
    color=["lightgray", "tomato"],
    figsize=(10, 6),
    width=0.8
)

# 7. Style
plt.title("🎭 Percentage of Reprogrammed Shows by Genre", fontsize=14)
plt.ylabel("Percentage")
plt.xlabel("Genre")
plt.xticks(rotation=45, ha='right')
plt.legend(["Not Reprogrammed", "Reprogrammed"])
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 1. Filter relevant data
df_viz = df.dropna(subset=["festival_theater", "reprog"])

# 2. Create counts per venue and reprog status
venue_counts = (
    df_viz.groupby(["festival_theater", "reprog"])
    .size()
    .unstack(level=1, fill_value=0)  # Make sure both 0 and 1 are present
)

# 3. Ensure both columns exist
for col in [0, 1]:
    if col not in venue_counts.columns:
        venue_counts[col] = 0

# 4. Normalize row-wise to percentages
venue_props = venue_counts.div(venue_counts.sum(axis=1), axis=0) * 100

# 5. Sort by proportion of reprog = 1
venue_props = venue_props.sort_values(by=1, ascending=False)

# 6. Plot
plt.figure(figsize=(12, 6))
venue_props[[0, 1]].plot(
    kind="bar",
    stacked=True,
    color=["lightgray", "tomato"],
    figsize=(12, 6),
    width=0.8
)

# 7. Style
plt.title("🎭 Percentage of Reprogrammed Shows by Venue", fontsize=14)
plt.ylabel("Percentage")
plt.xlabel("Venue (festival_theater)")
plt.xticks(rotation=45, ha='right')
plt.legend(["Not Reprogrammed", "Reprogrammed"])
plt.tight_layout()
plt.show()


### Inter-corelation

In [ ]:
# 1. Filter missing values
df_viz = df.dropna(subset=["genre_clean", "festival_theater"])

# 2. Create contingency table
contingency = pd.crosstab(df_viz["genre_clean"], df_viz["festival_theater"])

# 3. Run chi-squared test
chi2, p, dof, expected = chi2_contingency(contingency)

# 4. Compute Cramér’s V
n = contingency.sum().sum()
min_dim = min(contingency.shape) - 1
cramers_v = np.sqrt(chi2 / (n * min_dim))

# 5. Print results
print("📊 Chi-squared test for independence")
print(f"Chi² statistic = {chi2:.3f}")
print(f"p-value        = {p:.4g}")
print(f"Cramér’s V     = {cramers_v:.3f}")


In [ ]:
# 1. Filter missing values
df_viz = df.dropna(subset=["genre", "festival_theater"])

# 2. Create contingency table
contingency = pd.crosstab(df_viz["genre"], df_viz["festival_theater"])

# 3. Run chi-squared test
chi2, p, dof, expected = chi2_contingency(contingency)

# 4. Compute Cramér’s V
n = contingency.sum().sum()
min_dim = min(contingency.shape) - 1
cramers_v = np.sqrt(chi2 / (n * min_dim))

# 5. Print results
print("📊 Chi-squared test for independence")
print(f"Chi² statistic = {chi2:.3f}")
print(f"p-value        = {p:.4g}")
print(f"Cramér’s V     = {cramers_v:.3f}")

In [ ]:
df

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

# 1. Contingency table
contingency = pd.crosstab(df["festival_theater"], df["reprog"])

# 2. Chi-squared test
chi2, p, dof, expected = chi2_contingency(contingency)

# 3. Cramér’s V
n = contingency.sum().sum()
min_dim = min(contingency.shape) - 1
cramers_v = np.sqrt(chi2 / (n * min_dim))

# 4. Results
print("📊 Chi-squared test for independence")
print(f"Chi2 statistic = {chi2:.3f}")
print(f"p-value        = {p:.4f}")
print(f"Cramér’s V     = {cramers_v:.3f}")


### Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, confusion_matrix, ConfusionMatrixDisplay,
    roc_auc_score, RocCurveDisplay
)
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# ---------------------------------------------------------------
# 1️⃣  Train / test split  (80 % train, 20 % test, stratified)
# ---------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=42
)

# ---------------------------------------------------------------
# 2️⃣  Fit Logistic Regression
#    * class_weight="balanced" compensates for class-imbalance
#    * max_iter large enough to converge
# ---------------------------------------------------------------
clf = LogisticRegression(
    penalty="l2",
    solver="liblinear",
    class_weight="balanced",
    max_iter=2000,
)
clf.fit(X_train, y_train)

# ---------------------------------------------------------------
# 3️⃣  Evaluate on the hold-out test set
# ---------------------------------------------------------------
y_pred   = clf.predict(X_test)
y_proba  = clf.predict_proba(X_test)[:, 1]

print("\n📊 Classification report (test set):")
print(classification_report(y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=[0,1])
disp = ConfusionMatrixDisplay(cm, display_labels=["Not-prog (0)", "Prog (1)"])
disp.plot(cmap="Blues", values_format="d")
plt.title("Confusion Matrix (test)")
plt.show()

# ROC-AUC (optional)
auc = roc_auc_score(y_test, y_proba)
print(f"ROC-AUC: {auc:.3f}")
RocCurveDisplay.from_predictions(y_test, y_proba)
plt.show()

# ---------------------------------------------------------------
# 4️⃣  Feature importance  →  odds-ratios
# ---------------------------------------------------------------
odds_ratios = pd.Series(
    np.exp(clf.coef_[0]),
    index=X.columns
).sort_values(ascending=False)

print("\n🔥 Top positive effects (OR > 1):")
display(odds_ratios.head(20).round(3))

print("\n❄️ Top negative effects (OR < 1):")
display(odds_ratios.tail(20).round(3))


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# --- 1. Clean base data
df_clean = df.dropna(subset=[
    "duration_minutes", "n_actors", "genre_clean", "festival_theater", "reprog"
]).copy()

# Remove known outliers
df_clean = df_clean[~df_clean["duration_minutes"].isin([1440, 3300])]
df_clean["n_actors"] = pd.to_numeric(df_clean["n_actors"], errors="coerce")
df_clean["duration_minutes"] = pd.to_numeric(df_clean["duration_minutes"], errors="coerce")
df_clean = df_clean.dropna(subset=["n_actors", "duration_minutes"])

# --- 2. Create dummies
X = pd.get_dummies(
    df_clean[["n_actors", "duration_minutes", "genre_clean", "festival_theater"]],
    drop_first=True
).astype(float)

# --- 3. Remove columns with no variance or extreme sparsity
X = X.loc[:, X.nunique() > 1]
X = X.loc[:, X.sum() > 2]

# --- 4. Add constant manually
X = add_constant(X, has_constant='add')
y = df_clean["reprog"].astype(int)

# --- 5. Remove multicollinear columns using VIF (variance inflation factor)
def drop_high_vif(X, threshold=100):
    dropped_cols = []
    while True:
        vif = pd.Series(
            [variance_inflation_factor(X.values, i) for i in range(X.shape[1])],
            index=X.columns
        )
        max_vif = vif.drop("const").max()
        if max_vif > threshold:
            col_to_drop = vif.drop("const").idxmax()
            dropped_cols.append(col_to_drop)
            X = X.drop(columns=[col_to_drop])
        else:
            break
    return X, dropped_cols

X, dropped = drop_high_vif(X)
print("Dropped due to high VIF:", dropped)

# --- 6. Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# --- 7. Fit Probit model
probit_model = sm.Probit(y_train, X_train)
probit_result = probit_model.fit()

# --- 8. Predict + evaluate
y_proba = probit_result.predict(X_test)
threshold = 0.5
y_pred = (y_proba >= threshold).astype(int)

print(f"\n📊 Classification report (threshold = {threshold:.2f}):")
print(classification_report(y_test, y_pred))
print("\nConfusion matrix:")
print(confusion_matrix(y_test, y_pred))


### Random Forest

In [ ]:
file="/Users/antonioslagarias/Documents/OFF/Exports/adho/data_adho_prog.xlsx"  
data_draft=pd.read_excel(file)

In [ ]:
data_draft.columns

In [ ]:
pd.set_option("display.max_rows", None)  # Show all rows in output
print(data_draft["festival_theater"].value_counts(dropna=False))

In [ ]:
duration_minutes

In [ ]:
df = data_draft[["reprog", "n_actors", "duration_minutes",
                 "genre", "festival_theater"]].copy()

In [ ]:
df["n_actors"] = pd.to_numeric(df["n_actors"], errors="coerce")
df["duration_minutes"] = pd.to_numeric(df["duration_minutes"], errors="coerce")

In [ ]:
df['festival_theater'] = df['festival_theater'].fillna("Inconnu")

In [ ]:
genre_map = {
    # Théâtre / Café-théâtre
    "théâtre": "Théâtre / Café-théâtre", "café-théâtre": "Théâtre / Café-théâtre",
    "comédie": "Théâtre / Café-théâtre", "drame": "Théâtre / Café-théâtre",
    "boulevard": "Théâtre / Café-théâtre", "humour": "Théâtre / Café-théâtre",
    "théâtre sonore": "Théâtre / Café-théâtre",

    # Spectacle musical / Concert
    "théâtre musical": "Spectacle musical / Concert", "spectacle musical": "Spectacle musical / Concert",
    "concert": "Spectacle musical / Concert", "classique": "Spectacle musical / Concert",
    "expo-concert": "Spectacle musical / Concert", "chanson": "Spectacle musical / Concert",

    # Danse / Danse-théâtre
    "danse": "Danse / Danse-théâtre", "danse-théâtre": "Danse / Danse-théâtre",

    # Cirque / Clown
    "cirque": "Cirque / Clown", "clown": "Cirque / Clown",
    "magie": "Cirque / Clown", "théâtre musical/cirque": "Cirque / Clown",

    # Mime / Marionnettes-objets
    "mime": "Mime / Marionnettes-objets",
    "marionnette-objet": "Mime / Marionnettes-objets",
    "marionnette-objet de 7 mois à 4 ans": "Mime / Marionnettes-objets",

    # Conte / Poésie / Lecture
    "lecture": "Conte / Poésie / Lecture", "poésie": "Conte / Poésie / Lecture",
    "conte": "Conte / Poésie / Lecture", "théâtre-poésie": "Conte / Poésie / Lecture",
}

df["genre_clean"] = (
    df["genre"].str.lower().str.strip()
      .str.replace(r"\s*/\s*plein\s+air", "", regex=True)
      .map(genre_map)               # map to group or NaN
      .fillna("Other")              # unseen genres -> Other
)


In [ ]:
genre_dummies = pd.get_dummies(df["genre_clean"], prefix="genre", drop_first=True)
venue_dummies = pd.get_dummies(df["festival_theater"].fillna("Inconnu"),
                               prefix="venue", drop_first=True)
X = pd.concat([df[["n_actors", "duration_minutes"]],
               genre_dummies, venue_dummies], axis=1).fillna(0)

y = df["reprog"].astype(int)    # 0/1 target

print("✅ X shape:", X.shape)
print("✅ y distribution:", y.value_counts().to_dict())
X.head()

In [ ]:
pd.reset_option("display.max_rows")

In [ ]:
from imblearn.combine import SMOTETomek
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
# ─────────────────────────────────────────────────────────────
# 0. Imports  (run `pip install xgboost imbalanced-learn` if needed)
# ─────────────────────────────────────────────────────────────
from xgboost import XGBClassifier
from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report
from scipy.stats import uniform, randint
from collections import Counter
import numpy as np

# ─────────────────────────────────────────────────────────────
# 1. Train / test split
# ─────────────────────────────────────────────────────────────
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=42
)

print("Train class balance:", Counter(y_train))
print("Test  class balance:", Counter(y_test))

# ─────────────────────────────────────────────────────────────
# 2. Pipeline: SMOTETomek → XGBoost
# ─────────────────────────────────────────────────────────────
pipeline = Pipeline([
    ("smt", SMOTETomek(sampling_strategy="all", random_state=42)),
    ("xgb", XGBClassifier(
        objective="binary:logistic",
        eval_metric="logloss",
        random_state=320
    ))
])

# ─────────────────────────────────────────────────────────────
# 3. Parameter search space  (broad but not huge)
# ─────────────────────────────────────────────────────────────
param_dist = {
    "xgb__n_estimators"      : randint(150, 500),
    "xgb__max_depth"         : randint(3, 7),
    "xgb__learning_rate"     : uniform(0.01, 0.2),
    "xgb__subsample"         : uniform(0.6, 0.4),
    "xgb__colsample_bytree"  : uniform(0.6, 0.4),
    "xgb__gamma"             : uniform(0, 0.5),
    "xgb__reg_alpha"         : uniform(0, 0.2),
    "xgb__reg_lambda"        : uniform(0.1, 1.0),
    "xgb__scale_pos_weight"  : uniform(1.0, 3.0)   # imbalance tweak
}

# ─────────────────────────────────────────────────────────────
# 4. RandomizedSearchCV  (30 random combos × 5 folds)
# ─────────────────────────────────────────────────────────────
search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_dist,
    n_iter=30,                 # try 30 random combos
    cv=5,                      # 5-fold CV
    scoring="balanced_accuracy",
    n_jobs=-1,
    verbose=1,
    random_state=42
)

# ─────────────────────────────────────────────────────────────
# 5. Run search (may take a few minutes)
# ─────────────────────────────────────────────────────────────
search.fit(X_train, y_train)

print("\n🔎 Best hyper-parameters found:")
for k, v in search.best_params_.items():
    print(f"  {k}: {v}")

print(f"\n🏆 Best CV balanced-accuracy: {search.best_score_:.3f}")

# ─────────────────────────────────────────────────────────────
# 6. Evaluate best model on the hold-out test set
# ─────────────────────────────────────────────────────────────
best_model = search.best_estimator_
y_pred = best_model.predict(X_test)
print("\n📊 Classification report on test set:")
print(classification_report(y_test, y_pred))


In [ ]:
from xgboost import XGBClassifier
from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns, matplotlib.pyplot as plt
from collections import Counter

THRESH = 0.40                   # ← your chosen cutoff

# 1️⃣  Train / test split (same split every run)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=42
)

# 2️⃣  Pipeline: SMOTETomek → tuned XGBoost
pipe = Pipeline([
    ("smt", SMOTETomek(random_state=42, sampling_strategy="all")),
    ("xgb", XGBClassifier(
        n_estimators=250,
        max_depth=3,
        learning_rate=0.05,
        subsample=0.6,
        colsample_bytree=0.6,
        scale_pos_weight=1,          # keep 1.0 since threshold balances recall/precision
        eval_metric="logloss",
        objective="binary:logistic",
        random_state=320
    ))
])

pipe.fit(X_train, y_train)

# 3️⃣  Predict with custom threshold
proba = pipe.predict_proba(X_test)[:, 1]
y_pred = (proba >= THRESH).astype(int)

print(f"\n📊 Classification report (threshold = {THRESH}):")
print(classification_report(y_test, y_pred))

# 4️⃣  Confusion matrix for visual inspection
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap="Blues",
            xticklabels=["Pred 0", "Pred 1"],
            yticklabels=["True 0", "True 1"])
plt.title(f"Confusion Matrix at Threshold {THRESH}")
plt.show()


In [ ]:
from sklearn.metrics import f1_score, classification_report
import numpy as np
import matplotlib.pyplot as plt

# Predict probabilities for the positive class
y_proba = pipeline.predict_proba(X_test)[:, 1]

# Define range of thresholds
thresholds = np.linspace(0.01, 0.99, 99)
f1_scores = []

# Compute F1-score for each threshold
for t in thresholds:
    y_pred_t = (y_proba >= t).astype(int)
    score = f1_score(y_test, y_pred_t)
    f1_scores.append(score)

# Get best threshold
best_index = np.argmax(f1_scores)
best_thresh = thresholds[best_index]
best_f1 = f1_scores[best_index]

print(f"🏆 Best threshold for F1-score: {best_thresh:.2f}")
print(f"📈 Best F1-score: {best_f1:.4f}")

# Recompute predictions and report
y_best = (y_proba >= best_thresh).astype(int)
print("\n📊 Classification report (best threshold):")
print(classification_report(y_test, y_best))

# Optional: Plot F1 vs threshold
plt.figure(figsize=(6, 4))
plt.plot(thresholds, f1_scores, label="F1-score")
plt.axvline(best_thresh, color="red", linestyle="--", label=f"Best: {best_thresh:.2f}")
plt.xlabel("Threshold")
plt.ylabel("F1-score")
plt.title("F1-score vs Decision Threshold")
plt.grid()
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import shap
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# ─────────────────────────────────────────────────────────────
# 1. Extract the trained XGBoost model inside the pipeline
# ─────────────────────────────────────────────────────────────
xgb_model = best_model.named_steps["xgb"]          # adjust key if your step is called "classifier"

# ─────────────────────────────────────────────────────────────
# 2. Re-apply SMOTETomek so X matches what the model saw
# ─────────────────────────────────────────────────────────────
X_train_bal, y_train_bal = best_model.named_steps["smt"].fit_resample(X_train, y_train)

# It's often enough to sample 5 000 rows for speed
sample_idx = np.random.choice(len(X_train_bal), min(5000, len(X_train_bal)), replace=False)
X_sample = pd.DataFrame(X_train_bal.iloc[sample_idx], columns=X_train_bal.columns)

# ─────────────────────────────────────────────────────────────
# 3. Build the SHAP TreeExplainer
# ─────────────────────────────────────────────────────────────
explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer.shap_values(X_sample)

# ─────────────────────────────────────────────────────────────
# 4. GLOBAL: summary plot (beeswarm)
# ─────────────────────────────────────────────────────────────
shap.summary_plot(
    shap_values,
    X_sample,
    plot_type="dot",       # beeswarm
    max_display=20,        # top-20 features
    show=True              # pops up in notebook
)

# ─────────────────────────────────────────────────────────────
# 5. LOCAL: force plot for a single example (row 0)
# ─────────────────────────────────────────────────────────────
shap.initjs()  # enable JavaScript visualisation in notebook
i = 0          # pick any index in X_sample
shap.force_plot(
    explainer.expected_value,
    shap_values[i],
    X_sample.iloc[i],
    matplotlib=True        # static render (works inside JupyterLab)
)



In [ ]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

proba = pipe.predict_proba(X_test)[:, 1]
precision, recall, thresholds = precision_recall_curve(y_test, proba)

plt.plot(thresholds, precision[:-1], label="Precision")
plt.plot(thresholds, recall[:-1], label="Recall")
plt.xlabel("Threshold")
plt.ylabel("Score")
plt.legend()
plt.grid()
plt.title("Precision vs Recall for Class 1")
plt.show()


In [ ]:
# ────────────────────────────────────────────────────────────────
# 1) Copy only needed columns
# ----------------------------------------------------------------
df = data_draft[["reprog", "n_actors", "genre"]].copy()
df["reprog"] = df["reprog"].astype(int)

# ────────────────────────────────────────────────────────────────
# 2) Define the 6 custom genre groups
# ----------------------------------------------------------------
custom_genre_map = {
    "théâtre": "Théâtre / Café-théâtre",
    "café-théâtre": "Théâtre / Café-théâtre",
    "comédie": "Théâtre / Café-théâtre",
    "drame": "Théâtre / Café-théâtre",
    "boulevard": "Théâtre / Café-théâtre",
    "humour": "Théâtre / Café-théâtre",
    "théâtre sonore": "Théâtre / Café-théâtre",

    "lecture": "Conte / Poésie / Lecture",
    "poésie": "Conte / Poésie / Lecture",
    "conte": "Conte / Poésie / Lecture",
    "théâtre-poésie": "Conte / Poésie / Lecture",

    "danse": "Danse / Danse-théâtre",
    "danse-théâtre": "Danse / Danse-théâtre",

    "cirque": "Cirque / Clown",
    "clown": "Cirque / Clown",
    "théâtre musical/cirque": "Cirque / Clown",
    "magie": "Cirque / Clown",

    "mime": "Mime / Marionnettes-objets",
    "marionnette-objet": "Mime / Marionnettes-objets",
    "marionnette-objet de 7 mois à 4 ans": "Mime / Marionnettes-objets",

    "théâtre musical": "Spectacle musical / Concert",
    "spectacle musical": "Spectacle musical / Concert",
    "concert": "Spectacle musical / Concert",
    "classique": "Spectacle musical / Concert",
    "expo-concert": "Spectacle musical / Concert",
    "chanson": "Spectacle musical / Concert",
}

# Lowercase + strip to ensure match
df["genre_lower"] = df["genre"].str.lower().str.strip()

# Remove "/ plein air" etc. and normalize formatting
df["genre_lower"] = df["genre_lower"].str.replace(r"\s*/\s*plein\s+air", "", regex=True)

# Map to group label
df["genre_group"] = df["genre_lower"].replace(custom_genre_map)

# Drop any rows with unknown (unmapped) genres
df = df[df["genre_group"].notna()]

# ────────────────────────────────────────────────────────────────
# 3) One-hot encode grouped genres
# ----------------------------------------------------------------
genre_dummies = pd.get_dummies(df["genre_group"], prefix="genre", drop_first=True)

# ────────────────────────────────────────────────────────────────
# 4) Build final feature matrix and target
# ----------------------------------------------------------------
X = pd.concat([df[["n_actors"]], genre_dummies], axis=1).fillna(0)
y = df["reprog"]

# Check
print("✅ Prepared feature matrix:")
print("\nTarget distribution:\n", y.value_counts())
X.head()


In [ ]:
print("\n🎭 Genre group distribution:")
print(df["genre_group"].value_counts())

In [ ]:
X.columns

In [ ]:
# ────────────────────────────────────────────────────────────────
# 1. Train/test split BEFORE resampling
# ----------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# ────────────────────────────────────────────────────────────────
# 2. Apply SMOTE + Tomek links on training set
# ----------------------------------------------------------------
smt = SMOTETomek(sampling_strategy='all', random_state=320)
X_smt, y_smt = smt.fit_resample(X_train, y_train)

print("✅ After SMOTETomek:")
print(pd.Series(y_smt).value_counts())

# ────────────────────────────────────────────────────────────────
# 3. Train Random Forest
# ----------------------------------------------------------------
RandomForestClassifier(
    n_estimators=200,
    max_depth=5,              # limit tree depth
    min_samples_leaf=10,      # prevent tiny branches
    class_weight='balanced',  # handle imbalance
    random_state=42
)

clf.fit(X_smt, y_smt)

# ────────────────────────────────────────────────────────────────
# 4. Evaluate on unmodified test set
# ----------------------------------------------------------------
y_pred = clf.predict(X_test)
print("\n📊 Classification report on test set:")
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

probas = clf.predict_proba(X_test)[:, 1]
precision, recall, thresholds = precision_recall_curve(y_test, probas)

plt.plot(thresholds, precision[:-1], label='Precision')
plt.plot(thresholds, recall[:-1], label='Recall')
plt.xlabel("Threshold")
plt.ylabel("Score")
plt.title("Precision-Recall vs Threshold")
plt.legend()
plt.grid()
plt.show()

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(clf, X, y, cv=5, scoring="f1")
print("Mean F1-score:", scores.mean())

In [ ]:
from xgboost import XGBClassifier

clf = XGBClassifier(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.1,
    scale_pos_weight=4.5,  # try ratio: n_class_0 / n_class_1
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

In [ ]:
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
import pandas as pd

# ────────────────────────────────────────────────────────────────
# Optional: Calculate class imbalance ratio for weighting
# ----------------------------------------------------------------
ratio = (y == 0).sum() / (y == 1).sum()
print(f"Class imbalance ratio: {ratio:.2f}")

# ────────────────────────────────────────────────────────────────
# Define pipeline: SMOTE + XGBoost
# ----------------------------------------------------------------
pipeline = Pipeline([
    ('smote', SMOTE(random_state=42)),
    ('xgb', XGBClassifier(
        n_estimators=200,
        max_depth=5,
        learning_rate=0.1,
        scale_pos_weight=ratio,
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42
    ))
])

# ────────────────────────────────────────────────────────────────
# 5-fold cross-validation (F1 score)
# ----------------------------------------------------------------
scores = cross_val_score(pipeline, X, y, cv=5, scoring='f1')
print("\n✅ Mean F1-score (cross-validated):", scores.mean())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Fit full pipeline on training data
pipeline.fit(X_train, y_train)

# Predict on holdout test data
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
import matplotlib.pyplot as plt
from xgboost import plot_importance

# Access trained XGBoost model from pipeline
xgb_model = pipeline.named_steps["xgb"]

# Plot top features
plot_importance(xgb_model, max_num_features=10)
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_pred_thresh)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix (Threshold {:.2f})".format(threshold))
plt.show()

In [ ]:
from xgboost import XGBClassifier
from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter
import pandas as pd

# Split data (stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=42
)

for weight in [4.5, 3.0, 2.0, 1.0]:
    print(f"\n🧪 Testing scale_pos_weight = {weight}")

    # Apply SMOTETomek manually (before pipeline) to inspect resampled counts
    smt = SMOTETomek(sampling_strategy="all", random_state=42)
    X_train_smt, y_train_smt = smt.fit_resample(X_train, y_train)

    # Print class distribution before and after
    print("📊 Before SMOTETomek:", dict(Counter(y_train)))
    print("📊 After SMOTETomek: ", dict(Counter(y_train_smt)))

    # Now fit pipeline using the same resampled data
    clf = XGBClassifier(
        n_estimators=200,
        max_depth=5,
        learning_rate=0.1,
        scale_pos_weight=weight,
        use_label_encoder=False,
        eval_metric="logloss",
        random_state=42
    )
    clf.fit(X_train_smt, y_train_smt)

    # Evaluate on untouched test set
    y_pred = clf.predict(X_test)
    print(classification_report(y_test, y_pred))


In [ ]:
# ────────────────────────────────────────────────────────────────
# 0. Imports  (install xgboost / imbalanced-learn if needed)
#    !pip install xgboost imbalanced-learn
# ────────────────────────────────────────────────────────────────
from xgboost import XGBClassifier
from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from collections import Counter
import pandas as pd

# ────────────────────────────────────────────────────────────────
# 1. Train / test split  (stratified 80-20)
# ────────────────────────────────────────────────────────────────
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=42
)

print("Full dataset:", Counter(y))
print("Train split :", Counter(y_train))
print("Test  split :", Counter(y_test))

# ────────────────────────────────────────────────────────────────
# 2. Define SMOTETomek step  (random_state for reproducibility)
# ────────────────────────────────────────────────────────────────
smt = SMOTETomek(sampling_strategy="all", random_state=42)

# ────────────────────────────────────────────────────────────────
# 3. Base XGBoost model  (random_state=320 per your code)
# ────────────────────────────────────────────────────────────────
xgb_model = XGBClassifier(
    objective="binary:logistic",
    n_estimators=200,
    random_state=320,
    eval_metric="logloss",
)

# ────────────────────────────────────────────────────────────────
# 4. Build pipeline  (SMOTETomek ➜ XGBoost)
# ────────────────────────────────────────────────────────────────
pipeline = Pipeline([
    ("smt", smt),
    ("classifier", xgb_model)
])

# ────────────────────────────────────────────────────────────────
# 5. Parameter grid for grid-search
#    You can add/remove values to make the search finer/coarser.
# ────────────────────────────────────────────────────────────────
param_grid = {
    "classifier__max_depth"      : [3, 5, 7],
    "classifier__learning_rate"  : [0.1, 0.05, 0.01],
    "classifier__subsample"      : [0.6, 0.8, 1.0],
    "classifier__colsample_bytree": [0.6, 0.8, 1.0],
    "classifier__scale_pos_weight": [1.0, 2.0, 3.0],   # imbalance control
    "classifier__reg_alpha"      : [0, 0.01, 0.1],
    "classifier__reg_lambda"     : [1, 0.1, 0.01],
}

# ────────────────────────────────────────────────────────────────
# 6. GridSearchCV  (5-fold CV, balanced accuracy)
# ────────────────────────────────────────────────────────────────
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=5,
    scoring="balanced_accuracy",
    n_jobs=-1,                  # use all CPU cores
    verbose=1
)

# ────────────────────────────────────────────────────────────────
# 7. Fit grid search  (this may take a few minutes)
# ────────────────────────────────────────────────────────────────
grid_search.fit(X_train, y_train)

print("\n🔎 Best hyper-parameters:", grid_search.best_params_)
print("📈 Best CV balanced accuracy:", grid_search.best_score_)

# ────────────────────────────────────────────────────────────────
# 8. Inspect class distribution after SMOTETomek in best estimator
#    (fit_resample was already run inside CV; we run again for info)
# ────────────────────────────────────────────────────────────────
best_smt = grid_search.best_estimator_.named_steps["smt"]
_, y_train_bal = best_smt.fit_resample(X_train, y_train)
print("\nAfter SMOTETomek (train set):", Counter(y_train_bal))

# ────────────────────────────────────────────────────────────────
# 9. Evaluate best model on the untouched test set
# ────────────────────────────────────────────────────────────────
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("\n📊 Classification report on 25% hold-out test set:")
print(classification_report(y_test, y_pred))


In [ ]:
import shap

# Extract the trained XGBoost model from the pipeline
xgb_model = best_model.named_steps["classifier"]

# Use TreeExplainer for XGBoost
explainer = shap.TreeExplainer(xgb_model)

# Apply SMOTETomek to training data (just for matching the model)
X_train_bal, y_train_bal = best_model.named_steps["smt"].fit_resample(X_train, y_train)

# Compute SHAP values for a sample of data (can use whole set too)
shap_values = explainer.shap_values(X_train_bal)


In [ ]:
# Summary plot (global importance of all features)
shap.summary_plot(shap_values, X_train_bal)

In [ ]:
shap.initjs()
shap.force_plot(
    explainer.expected_value,
    shap_values[0],
    X_train_bal.iloc[0]
)


### Topic modelling

In [ ]:
from umap import UMAP
from bertopic import BERTopic

umap_model = UMAP(n_components=2, n_neighbors=3, min_dist=0.1)

topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    language="french"
)

In [ ]:
from collections import defaultdict
import operator

def get_top_words(mgp, topic, top_n=5):
    word_counts = mgp.cluster_word_distribution[topic]
    sorted_words = sorted(word_counts.items(), key=operator.itemgetter(1), reverse=True)
    return [word for word, count in sorted_words[:top_n]]

In [ ]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
import hdbscan
import numpy as np

# Sample French docs (very short)
docs = [
    "Nouvelle réforme du système éducatif.",
    "Les manifestations contre la loi ont repris.",
    "Découverte scientifique majeure en biologie.",
    "Hausse des prix de l'énergie en Europe.",
    "Les élections présidentielles approchent.",
]

# French-compatible embeddings
embedding_model = SentenceTransformer("Lajavaness/sentence-camembert-large")
embeddings = embedding_model.encode(docs, show_progress_bar=False)

# Fix UMAP for small dataset
umap_model = UMAP(n_neighbors=2, n_components=2, min_dist=0.1, metric="cosine")

# Fix HDBSCAN for small dataset
hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=2, min_samples=1, prediction_data=True)

# BERTopic with custom UMAP and HDBSCAN
topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    language="french",
    calculate_probabilities=True,
    verbose=True
)

# Fit the model
topics, probs = topic_model.fit_transform(docs, embeddings)

# Display topics
print("\n📌 Topic Summary:")
print(topic_model.get_topic_info())

print("\n🧠 Top words per topic:")
for topic_id in topic_model.get_topic_freq().Topic:
    if topic_id == -1:
        continue
    print(f"\nTopic {topic_id}:")
    for word, weight in topic_model.get_topic(topic_id):
        print(f"  {word:<15} {weight:.4f}")


In [ ]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

# Load French stopwords
french_stopwords = set(stopwords.words("french"))

# Extract and print labels
print("\n🏷️ Clean topic labels (top 2 non-stopwords):")
for topic_id in topic_model.get_topic_freq().Topic:
    if topic_id == -1:
        continue
    top_words = topic_model.get_topic(topic_id)
    
    # Filter out stopwords and empty strings
    keywords = [word for word, _ in top_words if word.lower() not in french_stopwords and word.strip() != ""]
    
    # Take the first two meaningful words
    label = " / ".join(keywords[:2])
    print(f"Topic {topic_id}: {label}")

In [ ]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
import hdbscan
import nltk
from nltk.corpus import stopwords

# Download stopwords once
nltk.download("stopwords", quiet=True)
FRENCH_STOPWORDS = set(stopwords.words("french"))

def build_topic_model(docs, n_neighbors=2, min_cluster_size=2):
    embedding_model = SentenceTransformer("Lajavaness/sentence-camembert-large")
    embeddings = embedding_model.encode(docs, show_progress_bar=False)

    umap_model = UMAP(n_neighbors=n_neighbors, n_components=2, min_dist=0.1, metric="cosine")
    hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size, min_samples=1, prediction_data=True)

    topic_model = BERTopic(
        embedding_model=embedding_model,
        umap_model=umap_model,
        hdbscan_model=hdbscan_model,
        language="french",
        calculate_probabilities=True,
        verbose=False
    )

    topic_model.fit(docs, embeddings)
    return topic_model

def get_clean_topic_labels(topic_model, top_n=2, stopwords_set=FRENCH_STOPWORDS):
    labels = {}
    for topic_id in topic_model.get_topic_freq().Topic:
        if topic_id == -1:
            continue
        keywords = [
            w for w, _ in topic_model.get_topic(topic_id)
            if w.lower() not in stopwords_set and w.strip()
        ]
        labels[topic_id] = " / ".join(keywords[:top_n])
    return labels


In [ ]:
data_draft.loc[10,"synopsis"]

In [ ]:
text = data_draft.loc[11, "synopsis"]
docs = [s.strip() for s in text.split(".") if len(s.strip().split()) > 4]

topic_model = build_topic_model(docs)
topic_labels = get_clean_topic_labels(topic_model)

print(docs)
for topic_id, label in topic_labels.items():
    print(f"Topic {topic_id}: {label}")

In [ ]:
from keybert import KeyBERT
kw_model = KeyBERT("paraphrase-multilingual-MiniLM-L12-v2")
text = data_draft.loc[10, "synopsis"]
keywords = kw_model.extract_keywords(text, top_n=5)
print(keywords)

In [ ]:
import yake
text = data_draft.loc[100, "synopsis"]

kw_extractor = yake.KeywordExtractor(lan="fr", n=1, top=5)
keywords = kw_extractor.extract_keywords(text)

for kw, score in keywords:
    print(f"{kw}: {score:.4f}")

### Structure production info

In [ ]:
def extract_production_metadata_gpt(input_string, api_key):

    prompt = build_production_prompt(input_string)
    client = openai.OpenAI(api_key=api_key)

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You extract structured metadata from production descriptions."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=1500,
            temperature=0
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
def build_production_prompt(production_text):
    return (
        "You will receive a string in French that may contain names of theaters and other cultural institutions that have participated in the making of a theater show."
        "There are three types of participation: 'production/corpoduction', 'funding' and 'other'."
        "Your task is to extract these data in JSON format:\n\n"
        "{\n"
        ' "coproduction": [...],'
        ' "funding": [...], '
        ' "other":[...] '
        ' "main":[...] \n '
        "}"
        "If it is 'production/coproduction', this will be indicated by key words such as 'produit', 'copoduction', 'coréalisation', etc. Return only the names of the institutions separated by ',' "
        "If it is 'funding', this will be indicated by key words such as 'financement', 'soutiens', 'avec le soutien de', 'avec l'aide de', etc. Return only the names of the institutions separated by ',' "
        "If you are not certain about where an institution should be put, add it 'other'. All institutions should be placed in a category. Keep the full name of each institutions (inclduing countries or other indicators)" 
        "If you detect is a press institution, ignore it. Ignore any names of people."
        "If you can clearly identify the institution that created the show add it in 'main' "
        "Do not return any explanation — only valid JSON.\n\n"
        f"Input:\n{production_text}"
    )

In [ ]:
def run_production_gpt_extraction(df, api_key, col="production_info", new_col="API_production", max_rows=None):
    
    subset = df if max_rows is None else df.iloc[:max_rows]
    results = []

    for i, row in subset.iterrows():
        txt = row[col]
        if isinstance(txt, str) and txt.strip():
            print(f"Processing row {i}…")
            resp = extract_production_metadata_gpt(txt, api_key)
            time.sleep(0.3)
        else:
            resp = None
        results.append(resp)

    # now assign in one shot—no chained assignment
    df[new_col] = pd.Series(results, index=subset.index)
    return df

In [ ]:
data_draft = run_production_gpt_extraction(data_draft,api_key)

In [ ]:
data_draft.loc[:, "API_production"] = data_draft["API_production"].apply(clean_json_string)

In [ ]:
data_draft